In [2]:
import os
import json
import pickle

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader

from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split

from collections import Counter

import sys
sys.path.append(os.pardir)
sys.path.append(os.path.join(os.pardir, "src"))
from src.models.multitask.multitask import *

_This notebook shows how the multitask BERT model used for landslide event extraction was trained._

## Loading dataset

_For more details about the dataset, please refer to the `multi-task-dataset-squad.ipynb` notebook for the `squad_extracted_questions.json` file and the `multi-task-dataset.ipynb` notebook for the `landslide_data.json` file._

In [3]:
DATA_PATH = os.path.join(
    os.path.dirname(os.path.abspath("__file__")), os.pardir, "data", "processed"
)
DATA_PATH

'/Users/badr/Documents/UBC/Classes/Capstone - COLX 595/BGC-NASA-landslide-detection/notebooks/../data/processed'

In [4]:
with open(os.path.join(DATA_PATH, "landslide_data.json")) as f:
       data = json.load(f)["data"]
len(data)

3216

In [5]:
with open(os.path.join(DATA_PATH, "squad_extracted_questions.json")) as f:
    squad_data = json.load(f)
len(squad_data)

12493

In [6]:
train_data, test_dev_data = train_test_split(data, test_size=0.20, random_state=123)
test_data, dev_data = train_test_split(test_dev_data, test_size=0.50, random_state=123)

len(train_data)

2572

In [22]:
# Save the splits
with open(os.path.join(DATA_PATH, "landslide_data_split.json"), "w") as f:
    split_data = {
        "version": "v1.0",
        "data": {"train": train_data, "dev": dev_data, "test": test_data},
    }
    json.dump(split_data, f)


In [7]:
squad_iter = LandslideEventsDataset(squad_data)
squad_dataloader = DataLoader(squad_iter, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
train_iter = LandslideEventsDataset(train_data)
dev_iter = LandslideEventsDataset(dev_data)
test_iter = LandslideEventsDataset(test_data)

train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, shuffle=True)
dev_dataloader = DataLoader(dev_iter, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_iter, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
train_data[2]

{'id': 18187,
 'text': 'Landslide hits traffic on Kodai road\nMADURAI: Incessant rain for the past few days in Kodaikanal hills have washed away a 50 feet road at Dum Dum Parai area, around seven km from the starting point of Ghat road section, on early hours of Monday.\n\nThe traffic was disrupted as the main road connecting Batlagundu in Dindigul district and Kodaikanal town got disconnected after the landslide. According to officials, it would take around 20 days to reconstruct the road as the damage was heavy.\n\nAfter the road was washed away, authorities from highways department swung into action and diverted Kodaikanal-bound vehicles through Pattiveeranpattti-Thandikudi road.\n\nThough the road, five kms from Dum Dum Parai, has got severely damaged by the landslide, there were no injuries, said officials.\n\nKodaikanal hills has been receiving heavy rain after around five years. A heavy rain that started at around 9pm on Sunday continued till midnight which resulted in landslide

In [10]:
squad_iter[0][3]

tensor([[63, 68],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0]])

In [11]:
train_iter[2][3]

tensor([[40, 44],
        [ 0,  0],
        [63, 64],
        [ 0,  0]])

## Training

### 1. Calculating Weights for Categorical Variables

In [14]:
cat_weights = [0.0] * len(CATEGORIES_ID2L)
trig_weights = [0.0] * len(TRIGGERS_ID2L)
# iob_weights = [0.0] * len(IOB_ID2L)

all_cat_ids = []
all_trig_ids = []
# all_iob_ids = []

for inputs, categories, triggers, spans, masks in train_iter:

    lengths = masks.sum(dim=-1)
    all_cat_ids.append(categories.item())
    all_trig_ids.append(triggers.item())
    # all_iob_ids.extend(iobs.tolist())

cat_total_count = len(all_cat_ids)
trig_total_count = len(all_trig_ids)
# iob_total_count = len(all_iob_ids)

cat_counter = Counter(all_cat_ids)
trig_counter = Counter(all_trig_ids)
# iob_counter = Counter(all_iob_ids)

for id, count in cat_counter.items():
    cat_weights[id] = 1 / (count / cat_total_count)

for id, count in trig_counter.items():
    trig_weights[id] = 1 / (count / trig_total_count)

# for id, count in iob_counter.items():
#     iob_weights[id] = 1 / (count / iob_total_count)

cat_weights = nn.functional.normalize(torch.tensor(cat_weights), p=1.0, dim=-1).to(
    DEVICE
)
trig_weights = nn.functional.normalize(torch.tensor(trig_weights), p=1.0, dim=-1).to(
    DEVICE
)
# iob_weights = nn.functional.normalize(torch.tensor(iob_weights), p=1.0, dim=-1).to(
#     DEVICE
# )


### 2. Pre-training on Squad dataset

In [19]:
LEARNING_RATE = 3e-5
EPOCHS = 2

model = LandslideEventsClassifier()
model.to(DEVICE)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    print(f"EPOCH: {epoch+1} of {EPOCHS}")
    epoch_loss = pre_train_model(squad_dataloader, model, optimizer)
    print(f"loss = {epoch_loss / len(squad_dataloader)}")


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


EPOCH: 1 of 2


100%|██████████| 1562/1562 [08:18<00:00,  3.13it/s]


loss = 10.667622141893023
EPOCH: 2 of 2


100%|██████████| 1562/1562 [08:21<00:00,  3.11it/s]

loss = 10.612337146006839


In [20]:
with torch.no_grad():
    cat_f1, trig_f1, span_acc = eval_model(dev_dataloader, model, True)
    print(
        f"category f1 score : {cat_f1} \ntrigger f1 score : {trig_f1} \nspan raw accuracy : {span_acc}"
    )

100%|██████████| 41/41 [00:04<00:00,  9.67it/s]

Classification report for landslide categories
                     precision    recall  f1-score   support

            complex       0.00      0.00      0.00         5
        debris_flow       0.00      0.00      0.00        12
         earth_flow       0.00      0.00      0.00         1
              lahar       0.00      0.00      0.00         1
          landslide       0.69      1.00      0.81       221
           mudslide       0.00      0.00      0.00        50
 riverbank_collapse       0.00      0.00      0.00         1
          rock_fall       0.00      0.00      0.00        28
   rotational_slide       0.00      0.00      0.00         1
translational_slide       0.00      0.00      0.00         1

           accuracy                           0.69       321
          macro avg       0.07      0.10      0.08       321
       weighted avg       0.47      0.69      0.56       321

Classification report for landslide triggers
                   precision    recall  f1-score   

In [21]:
c, t, s = model(list(dev_dataloader)[0][0].to(DEVICE), list(dev_dataloader)[0][-1].to(DEVICE))
pred_spans = select_best_answer_span(s[0][0], s[0][1], 5)

for i, pred_s in enumerate(pred_spans):
    print(TOKENIZER.decode(list(dev_dataloader)[0][0][i][pred_s[0] : pred_s[1]]))

pred_spans = select_best_answer_span(s[1][0], s[1][1], 5)

for i, pred_s in enumerate(pred_spans):
    print(TOKENIZER.decode(list(dev_dataloader)[0][0][i][pred_s[0] : pred_s[1]]))

 Vanuatu
 Putra Heights
 Yasamal district
 relief camps
 east of Canoe
 Cuthbert Road
 Hagi
 backyards










### 3. Training on NASA dataset

#### First 20 epochs

In [22]:
LEARNING_RATE = 3e-5
EPOCHS = 20

for epoch in range(EPOCHS):
    print(f"EPOCH: {epoch+1} of {EPOCHS}")
    epoch_loss = train_model(train_dataloader, model, optimizer)
    print(f"loss = {epoch_loss / len(train_dataloader)}")

    if epoch % 5 == 0:
        cat_f1, trig_f1, span_acc = eval_model(dev_dataloader, model)
        print(
            f"epoch : {epoch} \ncategory f1 score : {cat_f1} \ntrigger f1 score : {trig_f1} \nspan raw accuracy : {span_acc}"
        )

EPOCH: 1 of 20


100%|██████████| 322/322 [01:44<00:00,  3.08it/s]


loss = 47.80444531411118


100%|██████████| 41/41 [00:04<00:00,  9.66it/s]


epoch : 0 
category f1 score : 0.7009345794392523 
trigger f1 score : 0.34579439252336447 
span raw accuracy : 0.4588509500026703
EPOCH: 2 of 20


...

loss = 42.737167239929576
EPOCH: 19 of 20


100%|██████████| 322/322 [01:44<00:00,  3.08it/s]


loss = 42.660405117532484
EPOCH: 20 of 20


100%|██████████| 322/322 [01:44<00:00,  3.08it/s]

loss = 42.6323239699654


In [23]:
with torch.no_grad():
    cat_f1, trig_f1, span_acc = eval_model(dev_dataloader, model, True)
    print(
        f"category f1 score : {cat_f1} \ntrigger f1 score : {trig_f1} \nspan raw accuracy : {span_acc}"
    )


100%|██████████| 41/41 [00:04<00:00,  9.69it/s]

Classification report for landslide categories
                     precision    recall  f1-score   support

            complex       0.25      0.20      0.22         5
        debris_flow       0.00      0.00      0.00        12
         earth_flow       0.00      0.00      0.00         1
              lahar       0.00      0.00      0.00         1
          landslide       0.89      0.86      0.88       221
           mudslide       0.54      0.74      0.62        50
 riverbank_collapse       0.00      0.00      0.00         1
          rock_fall       0.66      0.68      0.67        28
   rotational_slide       0.00      0.00      0.00         1
translational_slide       0.00      0.00      0.00         1

           accuracy                           0.77       321
          macro avg       0.23      0.25      0.24       321
       weighted avg       0.76      0.77      0.76       321

Classification report for landslide triggers
                         precision    recall  f1-sc

In [24]:
with open("./models/landslide_detection-QA-2-epoch_2-20.model", "wb") as f:
    pickle.dump(model, f)

#### Second 20 epochs

In [25]:
LEARNING_RATE = 3e-5
EPOCHS = 20

for epoch in range(EPOCHS):
    print(f"EPOCH: {epoch+1} of {EPOCHS}")
    epoch_loss = train_model(train_dataloader, model, optimizer)
    print(f"loss = {epoch_loss / len(train_dataloader)}")

    if epoch % 5 == 0:
        cat_f1, trig_f1, span_acc = eval_model(dev_dataloader, model)
        print(
            f"epoch : {epoch} \ncategory f1 score : {cat_f1} \ntrigger f1 score : {trig_f1} \nspan raw accuracy : {span_acc}"
        )

EPOCH: 1 of 20


100%|██████████| 322/322 [01:44<00:00,  3.08it/s]


loss = 42.61324164586038


100%|██████████| 41/41 [00:04<00:00,  9.70it/s]


epoch : 0 
category f1 score : 0.7881619937694704 
trigger f1 score : 0.45482866043613707 
span raw accuracy : 0.40799689292907715
EPOCH: 2 of 20


...

loss = 42.211145602398034
EPOCH: 19 of 20


100%|██████████| 322/322 [01:44<00:00,  3.08it/s]


loss = 42.21656413107925
EPOCH: 20 of 20


100%|██████████| 322/322 [01:44<00:00,  3.08it/s]

loss = 42.23065338371703


In [28]:
with torch.no_grad():
    cat_f1, trig_f1, span_acc = eval_model(dev_dataloader, model, True)
    print(
        f"category f1 score : {cat_f1} \ntrigger f1 score : {trig_f1} \nspan raw accuracy : {span_acc}"
    )


100%|██████████| 41/41 [00:04<00:00,  9.71it/s]

Classification report for landslide categories
                     precision    recall  f1-score   support

            complex       0.50      0.40      0.44         5
        debris_flow       0.00      0.00      0.00        12
         earth_flow       0.00      0.00      0.00         1
              lahar       0.00      0.00      0.00         1
          landslide       0.89      0.89      0.89       221
           mudslide       0.62      0.84      0.71        50
 riverbank_collapse       0.00      0.00      0.00         1
          rock_fall       0.64      0.64      0.64        28
   rotational_slide       0.00      0.00      0.00         1
translational_slide       0.00      0.00      0.00         1

           accuracy                           0.80       321
          macro avg       0.27      0.28      0.27       321
       weighted avg       0.78      0.80      0.79       321

Classification report for landslide triggers
                         precision    recall  f1-sc

In [11]:
with open("./models/landslide_detection-QA-2-epoch_2-40.model", "wb") as f:
    pickle.dump(model, f)

# with open("./models/landslide_detection-QA-2-epoch_2-40.model", "rb") as f:
#     model = pickle.load(f)

#### Testing trainined model

In [19]:
batch_id = 10
c, t, s = model(list(dev_dataloader)[batch_id][0].to(DEVICE), list(dev_dataloader)[batch_id][-1].to(DEVICE))
for span_id in range(len(SPAN_ID2L)):
    print(SPAN_ID2L[span_id])
    pred_spans = select_best_answer_span(s[span_id][0], s[span_id][1], 5)
    
    for i, pred_s in enumerate(pred_spans):
        print(TOKENIZER.decode(list(dev_dataloader)[batch_id][0][i][pred_s[0] : pred_s[1]]).strip())
    print("\n")

LOC
Plomari
Bothell
Warren County
Hillgrove
Digdool
Monkayo
Asturias
Goehring Street


TIME

morning
this evening
last night
Monday night
Friday evening
morning



DATE

Tuesday morning
the past week

Tuesday
Sunday
yesterday



CAS


two


four






In [24]:
predict(
    model,
    [
        "Assam Rains: The state has recorded 42 deaths this year from floods and landslides. (File)\n\nGuwahati:\n\nFresh landslides were reported from various parts of Guwahati on Wednesday, amid incessant rainfall, which also led to heavy flooding in most parts of the city, officials said.\n\nDuring the day there were no reports of new casualties.\n\nThe state has recorded 42 deaths this year from flooding and landslides, including four that were buried under a massive mound of earth in the Boragaon area on Tuesday.\n\nAn official of the Assam State Disaster Management Authority (ASDMA) told PTI that accumulation of debris caused by landslides has blocked passageways in the Geetanagar, Sonapur, Kalapahar and Nijarapar areas.\n\nHeavy rainfall has also flooded roads in several parts of the city – the worst affected among them are Anil Nagar, Nabin Nagar, Rajgarh Link Road, Rukminigaon, Hatigaon and Krishna Nagar.\n\nNDRF and SDRF personnel, deployed in these areas, are using boats to rescue people and offer relief supplies to those still trapped in flooded homes, the official said.\n\nAssam Power Distribution Company Limited (APDCL) is working around the clock to restore power in the city, which has been in darkness since Tuesday, while drinking water tankers have been sent to the flooded areas.\n\nThe Kamrup Metropolitan district administration has urged people to leave their homes only if they have urgent or essential work.\n\nKamrup Metropolitan Deputy Commissioner Pallav Gopal Jha has issued an order ordering all schools, colleges and educational institutions to suspend classes in the wake of a rain alert issued by the Regional Meteorological Center (RMC).\n\nThe RMC issued a ‘red alert’ for Assam and Meghalaya and forecast isolated and scattered ‘extremely heavy rain’ from Tuesday to Thursday, and ‘orange alert’ for Friday and Saturday.\n\nA bulletin issued by the ASDMA said the flooding situation in several parts of the state is grim, with eight districts being inundated by rising waters in the Brahmaputra River and its tributaries.\n\nDima Hasao Deputy Commissioner Nazreen Ahmed has issued an advisory asking people to restrict movement in public places.\n\nThe consultancy has urged people in vulnerable locations to go to the nearest notified shelters.\n\nNazreen Ahmed has issued another order ordering all educational institutions to remain closed until May 17, an official release said.\n\nThe mighty Brahmaputra flows above the danger sign at Nimaathat in Jorhat district.\n\nAccording to officials, levees, roads, bridges and other infrastructure have been damaged by flooding in Barpeta, Cachar, Chirang, Dhemaji and Goalpara, among others.\n\nMeanwhile, state congress president Bhupen Boarh led a rally of party members here to protest the state government’s alleged apathy over the situation caused by flash floods in the ‘smart city’.\n\nThe protesters, who wade through knee-deep water, were stopped by police from continuing at a distance from the state party headquarters.\n\n(Except for the headline, this story has not been edited by DailyExpertNews staff and has been published from a syndicated feed.)",
        "Heavy rain in China has claimed six lives this week and forced the evacuation of some 200,000 people\n\nBEIJING -- Heavy rain in China has claimed six lives this week and forced the evacuation of some 200,000 people.\n\nFive people died in southeastern China, according to state media reports: A house collapse took three lives in Fujian province on Tuesday, and two died Monday in a landslide caused by torrential rain in neighboring Jiangxi province.\n\nOne person died and three are missing in northern China after a vehicle fell into the water in mountain flooding from heavy rain in the Inner Mongolia region.\n\nChina regularly experiences seasonal rains and flooding during the summer, most frequently in central and southern areas.\n\nVideo on state broadcaster CCTV showed large parts of cities flooded in muddy waters. Rescue workers carried trapped people out of buildings on their backs in waist-deep water in Guangdong province, also in the southeast.\n\nThe rain ended Wednesday in Jiangxi, but more precipitation is forecast for another week in Fujian, Jiangxi and Guangdong and the nearby Guangxi region. The National Meteorological Center said residents should prepare for floods and landslides as the soil is already saturated in those areas.\n\nMore than 110,000 people have been relocated in Jiangxi province, along with more than 80,000 in Fujian, state media reports said.\n\nSeventeen people died last week in rain-related disasters in Guangxi and in Hunan province in central China.",
        "BEIJING (AP) — Heavy rain in China has claimed six lives this week and forced the evacuation of some 200,000 people.\n\nFive people died in southeastern China, according to state media reports: A house collapse took three lives in Fujian province on Tuesday, and two died Monday in a landslide caused by torrential rain in neighboring Jiangxi province.\n\nOne person died and three are missing in northern China after a vehicle fell into the water in mountain flooding from heavy rain in the Inner Mongolia region.\n\nChina regularly experiences seasonal rains and flooding during the summer, most frequently in central and southern areas.\n\nVideo on state broadcaster CCTV showed large parts of cities flooded in muddy waters. Rescue workers carried trapped people out of buildings on their backs in waist-deep water in Guangdong province, also in the southeast.\n\nThe rain ended Wednesday in Jiangxi, but more precipitation is forecast for another week in Fujian, Jiangxi and Guangdong and the nearby Guangxi region. The National Meteorological Center said residents should prepare for floods and landslides as the soil is already saturated in those areas.\n\nMore than 110,000 people have been relocated in Jiangxi province, along with more than 80,000 in Fujian, state media reports said.\n\nSeventeen people died last week in rain-related disasters in Guangxi and in Hunan province in central China.\n\nRead more about:\n\nSHARE:",
        "https://www.nps.gov/yell/learn/news/220613.htm (This will refresh and will work as a link you can save and not navigate through the page to get to alerts)\n\nAS OF 11:07 Central Time This is what I am seeing (Unedited - I copy and pasted. Formatting may get weird) CLICK THE LINK\n\nTHIS DOES NOT MEAN EVERYTHING ELSE WILL OPEN. This means - North side? OH... it's bad. South side? We will let you know. More WILL come in the coming days.\n\nPeople keep asking for sources - here is one for you to save\n\nUPDATE: Northern portion of Yellowstone National Park likely to remain closed for a substantial length of time due to severely damaged, impacted infrastructure\n\nVisitors traveling to park in coming weeks must stay informed about current situation, road and weather conditions\n\nUPDATE: June 14 at 6:38 p.m.Northern portion of Yellowstone National Park likely to remain closed for a substantial length of time due to severely damaged, impacted infrastructure; Visitors traveling to park in coming weeks must stay informed about current situation, road and weather conditions\n\nUpdates\n\nAerial assessments conducted Monday, June 13, by Yellowstone National Park show major damage to multiple sections of road between the North Entrance (Gardiner, Montana), Mammoth Hot Springs, Lamar Valley and Cooke City, Montana, near the Northeast Entrance.\n\nMany sections of road in these areas are completely gone and will require substantial time and effort to reconstruct.\n\nThe National Park Service will make every effort to repair these roads as soon as possible; however, it is probable that road sections in northern Yellowstone will not reopen this season due to the time required for repairs.\n\nTo prevent visitors from being stranded in the park if conditions worsen, the park in coordination with Yellowstone National Park Lodges made the decision to have all visitors move out of overnight accommodations (lodging and campgrounds) and exit the park.\n\nAll entrances to Yellowstone National Park remain temporarily CLOSED while the park waits for flood waters to recede and can conduct evaluations on roads, bridges and wastewater treatment facilities to ensure visitor and employee safety.\n\nThere will be no inbound visitor traffic at any of the five entrances into the park, including visitors with lodging and camping reservations, until conditions improve and park infrastructure is evaluated.\n\nThe park’s southern loop appears to be less impacted than the northern roads and teams will assess damage to determine when opening of the southern loop is feasible. This closure will extend minimally through next weekend.\n\nDue to the northern loop being unavailable for visitors, the park is analyzing how many visitors can safely visit the southern loop once it’s safe to reopen. This will likely mean implementation of some type of temporary reservation system to prevent gridlock and reduce impacts on park infrastructure.\n\nAt this time, there are no known injuries nor deaths to have occurred in the park as a result of the unprecedented flooding.\n\nEffective immediately, Yellowstone’s backcountry is temporarily closed while crews assist campers (five known groups in the northern range) and assess damage to backcountry campsites, trails and bridges.\n\nThe National Park Service, surrounding counties and states of Montana and Wyoming are working with the park’s gateway communities to evaluate flooding impacts and provide immediate support to residents and visitors.\n\nWater levels are expected to recede today in the afternoon; however, additional flood events are possible through this weekend.\n\nKnown damage and issues\n\nKnown damage (at this time) to some park roads includes: North Entrance (Gardiner, Montana) to Mammoth Hot Springs: road washed out in multiple places, significant rockslide at Gardner Canyon Tower Junction to Northeast Entrance: segment of road washed out near Soda Butte Picnic Area, mudslides, downed trees Tower-Roosevelt to Canyon Junction (Dunraven Pass): mudslide on road Canyon Junction to Fishing Bridge: Segment of road just south of Canyon Junction potentially compromised and closed for evaluation\n\nThe power continues to be out in multiple locations in the park.\n\nWater and wastewater systems at Canyon Village and Mammoth Hot Springs are being impacted by flooding conditions and are being monitored.\n\nStay informed",
        "Landslides involve the movement of a large mass of earth down a slope and they can be caused by various factors including poor soil structure, rainfall, drought, erosion, and seismic activities (earthquakes). An avalanche is the movement of large snow volumes.\n\nWhat is the process of soil creep?\n\nSoil creep is a very slow movement, occurring on very gentle slopes because of the way soil particles repeatedly expand and contract in wet and dry periods. When wet, soil particles increase in size and weight, and expand at right angles. … As a result, the soil slowly moves down slope.\n\nIs landslide a calamity?\n\nLandslides occur when ground on slopes becomes unstable. … Given the nature of such disasters, fast relief and rescue operations are required to get to trapped survivors but this can be delayed due to landslides cutting off easy access to the affected area.\n\nWhat is soil creep?\n\nSoil creep defines the slow mass wasting process of soil on a slope, under the influence of gravity (Source: Glossary of Soil Science terms, Soil Science Society of America). … It was also stated that the majority of rates of solifluction affected soil movement down to 50 cm of depth.\n\nWhat is an example of a soil creep?\n\nOne example of soil creep in real life is from the wealthy Caribbean nation of Trinidad and Tabogo. … The movement of the soil caused instability and triggered other, more dramatic forms of mass wasting such as debris flows, and the road required reconstruction to combat the one-inch-per-year creep in the area.\n\nWhat causes creep movement?\n\nCreep is the imperceptibly slow, steady, downward movement of slope-forming soil or rock. Movement is caused by shear stress sufficient to produce permanent deformation, but too small to produce shear failure.\n\nWhat are 4 types of landslides?\n\nLandslides are part of a more general erosion or surficial pro- cess known as mass wasting, which is simply the downslope movement of earth or surface materials due to gravity. They are classified into four main types: fall and toppling, slides (rotational and translational), flows and creep.\n\nWhat causes landslide?\n\nLandslides are caused by disturbances in the natural stability of a slope. They can accompany heavy rains or follow droughts, earthquakes, or volcanic eruptions. Mudslides develop when water rapidly accumulates in the ground and results in a surge of water-saturated rock, earth, and debris.\n\nHow can human activities trigger landslide?",
        "Tamenglong/Noney: Massive landslides at numerous places and washed away culverts along the Imphal-Jiribam route completely cut off the National Highway 37 on Saturday, leaving many travellers stranded along this lifeline for the people of Manipur. However, no casualties or injuries have been reported from the incident.\n\nThe NH-37 remains cut off after a major portion of the road caved in between Khongsang and Irang in Noney district due to incessant rainfall over the last few days.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nVehicles plying between Nungba and Puiluan were stranded after the culvert at Chumla stream was washed away following a continuous downpour.\n\nAnother massive landslides has been reported from the old Cachar road near Joujangtek, Bwanruang Taodaijang and Kotlen between Imphal and Noney. With the intervention of local MLA and HAC chairman Dinganglung (Dipu) Gangmei, clearing of roads has now started along the affected highway.\n\nMany heavy trucks coming from Jiribam are stranded at Tamenglong with numerous landslides reported along the highway. Officials said the affected road between Tamenglong and Imphal is expected to clear by Sunday through the old Cachar road.\n\nAlso Read | Manipur: Security forces seize narcotics worth Rs 2.5 crore\n\nTrending Stories\n\n\n\n\n\n\n\n\n\n",
        "By Express News Service\n\nGUWAHATI: Twenty people died in the rain-induced floods and landslides in Assam and Meghalaya on Friday. While seven persons died in Assam, the remaining 13 were residents of Meghalaya. “As per flood data till 8 pm, 22 districts and 11,03,026 people have been affected. Seven persons died,” the Assam State Disaster Management Authority said.\n\nWith these, 52 persons have lost their lives in floods and landslides in the state since May. Cropland in 20,141.99 hectares has been affected. Rivers such as Brahmaputra, Kopili, Jia Bharali, Puthimari, Manas and Beki are in a spate, flowing above the danger level. Several areas of Guwahati, Rangiya and Nalbari remain inundated.\n\nThe National Disaster Response Force and the State Disaster Response Force, among other agencies, are engaged in the rescue and evacuation of the marooned people. In Meghalaya, 13 persons lost their lives in landslides. The deaths were reported from three districts – East Khasi Hill, South West Khasi Hills and South Garo Hills. In one of the incidents, three members of a family, including a minor, lost their lives.\n\nSeveral roads are either damaged or have been washed away. The movement of vehicles from Assam’s Brahmaputra valley to Barak valley has been disrupted following landslides in Meghalaya through which the road passes. The Regional Meteorological Centre has issued a warning for thunderstorm with lightning and extremely heavy rain at isolated places in parts of the Northeast.\n\nHighest single-day rainfall in June\n\nGuwahati: Mawsynram in Meghalaya on Friday recorded the highest all-time single-day rainfall in June. At 1,003.6 mm, it eclipsed the previous record of 945.4 mm measured on June 7, 1966. Cherrapunjee on Friday recorded the third highest rainfall of 972 mm in June. The all-time highest rainfall (1,563.3 mm) here was recorded on June 16, 1995 and the all-time second highest (973.8 mm) rainfall was recorded on June 5, 1956. According to the RMC, Mawsynram gets higher rainfall due to its higher elevation and location on crest of a hill on the edge of a valley opening towards the south.",
        "Shillong: Meghalaya administration is on alert across the state following incessant rains, which led to several landslides on one of the crucial lifelines—National Highway 6—under the Lumshnong Police Station under East Jaintia Hills.\n\nThe disruption in NH-6 will also cause inconvenience to states like Mizoram and Tripura along with Barak Valley in southern Assam.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nMeghalaya Chief Minister Conrad K Sangma held a review meeting on Thursday with deputy commissioners of different districts, following which they decided to form four regional committees, which will be headed by a minister.\n\n“I have asked them to ensure steps are taken so that the movement of vehicles can start at the earliest, especially of the essential supplies required not just for these districts, but for neighbouring states also,” he said.\n\nEarlier, social media was flooded with visuals of a truck along with a car stuck in a pit after a road caved in. Thankfully, no casualties were reported.\n\nThe East Jaintia Hills Deputy Commissioner, Abhilash Baranwal also issued a public notice which reads, “Due to continuous heavy rainfall over the last week, NH 06 has been damaged at multiple locations across the district especially from Nongsning to Ratacherra stretch. A section of NH6 near the Lumshnong Toll Plaza has also been washed away.”\n\nThe DC has advised avoiding unnecessary travel along the stretch. “In cases of emergency also, they should observe utmost caution while travelling through NH 06 in that section till further update about the restoration of the road,” advised the DC.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nThe Indian Meteorological Department (IMD) Shillong has issued a red alert for South West Khasi Hills, East Khasi Hills, West Jaintia and East Jaintia Hills districts.\n\nLahkmen Rymbui, home minister along with the disaster management minister Kyrmen Shylla will monitor East and West Jaintia hills; deputy chief minister Prestone Tynsong will head the monitoring committee for Ri-Bhoi and for East Khasi Hills District; Eastern West Khasi hills, West Khasi Hills, and South West Khasi Hills will be monitored by PHE minister Renikton Tongkhar. Finally, power minister James Sangma will monitor the entire Garo Hills region, which consists of North, East, South West, South and West Garo Hills Districts.\n\nThe regional committees will monitor the situation closely and ensure the movement and provision of essential commodities.\n\nThe committee must also visit their areas to ensure proper coordination between various departments and district officials, CM Sangma added.\n\nThe government will be monitoring the situation closely for the next 24 hours to 48 hours and another review meeting will be held tomorrow to take stock of the situation.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nAlso read: Meghalaya: Air Marshal Dilip Patnaik says Agnipath scheme to benefit youth\n\nTrending Stories\n\n\n\n\n\n\n\n\n\n",
        "Already battling heavy rains, flash floods, landslides and more calamity over the past week, all of Northeast India is likely to see more inclement weather in the coming days, the weatherman has said.\n\nThe Indian Meteorological Department has forecast extremely heavy rains over parts of Assam, Meghalaya, Sub-Himalayan West Bengal and Sikkim. They also predict heavy rains over other parts of the Northeast, including Arunachal Pradesh, Nagaland, Manipur, Mizoram, and Tripura.\n\nADVERTISEMENT CONTINUE READING BELOW\n\n“Widespread thundershowers with isolated heavy to very heavy rainfall very likely over northeast India and adjoining Sub-Himalayan West Bengal & Sikkim during next 5 days. Isolated extremely heavy rainfall also likely over Assam & Meghalaya during 16th -18th June; over Arunachal Pradesh on 16th and over Sub-Himalayan West Bengal & Sikkim on 16 th & 17th June, 2022,” the IMD said in a press release on June 16.\n\nThe IMD also said heavy showers will lash Gangetic West Bengal and Jharkhand.\n\nAccording to data, Assam received 192.6 mm of rainfall in the last week ending Wednesday against a normal average of 96.1 mm this time of the year – recording about 100% above normal rainfall.\n\nIncessant rains have wreaked havoc in various parts of Assam, with several regions in lower and central Assam on high alert. Hojai district in Central Assam is on alert after water levels in the Kopili river rose significantly in the past 24 hours, especially after water was released from a dam in the hill region of Karbi Anglong. The authorities have issued an alert across various districts and have strictly advised people to not venture out unless necessary.\n\nThe situation in Meghalaya seems to be worsening too, with road services snapped in various places due to landslides.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nSohra, also known as Cherrapunji—one of the wettest places in the world—recorded a massive 811.6 mm of rainfall in 24 hours ending 8:30 am on June 15, the highest in June since 1995.\n\nCherrapunji has recorded more than 750 mm of precipitation on a June day on 10 occasions since the IMD started keeping records, the IMD data showed.\n\nFresh spell of heavy rains in rest of Meghalaya, Mizoram and Tripura has caused several landslides on National Highway 06 near Meghalaya’s Lumshnong area that connects Tripura and Mizoram with the rest of the country.\n\nMassive crisis of essential supplies is also likely to hit several Northeastern states, including Tripura, Meghalaya, Assam, Arunachal and Mizoram.\n\nAlso Read | Meghalaya: Landslides damage NH-6; supplies to Tripura, Mizoram hit\n\nADVERTISEMENT CONTINUE READING BELOW\n\nTrending Stories\n\n\n\n\n\n\n\n\n\n",
        "What Causes A Rockslide?\n\nA rockslide is a type of landslide caused by rock failure in which part of the bedding plane of failure passes through compacted rock and material collapses en masse and not in individual blocks. … The rocks tumble downhill, loosening other rocks on their way and smashing everything in their path.\n\nWhat can trigger a rockslide?\n\nHigh precipitation, natural erosion, temperature variations or extreme stresses such as earthquakes can trigger rock slides or rock flows. When infrastructure and buildings are hit, the results are often catastrophic.\n\nWhat are the causes of landslide?\n\nLandslides are caused by disturbances in the natural stability of a slope. They can accompany heavy rains or follow droughts, earthquakes, or volcanic eruptions. Mudslides develop when water rapidly accumulates in the ground and results in a surge of water-saturated rock, earth, and debris.\n\nWhere do rock slides happen?\n\nA rock slide is a type of landslide occurring when a mass of rock moves quickly downslope. Rock slides happen in mountainous regions or where artificial excavation is taking place (e.g., mines and quarries).\n\nHow does a rock avalanche occur?",
        "(His Capture)\n\nIt was just after midnight when we got set upon. We were making up for lost time after routing around a rockslide that collapsed the wider path. We’d never seen any of them this far East. Hell, we were still a week away from the contested areas, and no message had made it to us that anything was different. We had some pressure on the outskirts these last few months but there was nothing like this. When we heard the horn, we all turned like good, ready warriors and readied to end the lives of any who would come against us. Then we heard the footsteps. Then we felt the footsteps. After the line split down the middle from the first wave, we heard the call to retreat. It all went to hell so fast. Not even a minute before we had no idea they were even there. Now half my clan lied dead in the mud as we started falling back, down the mountainside. The bastards were everywhere. We would cut one down, but two would swarm in behind before their blood even hit the ground. They were savage and undisciplined, but they just had the numbers. I couldn’t tell you if they were in the hundreds, but they felt like it. My mount took a spear in the hindquarters and buckled. I’ve never hit the ground so hard in my life. Damn near felt like my spine would snap as Ebben rolled onto, and then over me. I didn’t see it, but I heard him hit the trees behind me. He got a terrified scream out before another hard hit cut it short. After that I just heard his weight rolling down the mountainside.\n\nI went into a tree myself. It kept me from spilling down a steep embankment, but it cost me a few ribs and the ability to breath for what felt like an eternity, but was only probably ten seconds or so. The only reason I am alive is that they got to me before I could get back to my feat. The tree had actually saved me twice over. Yet here I was now, face down with a knee into my freshly broken ribs, and that air air was gone again. Even through the tunnel vision, I could hear and feel the metal getting closed around my wrists, and even though I felt like I was about to die, I knew I wouldn’t, but that I would awaken enslaved. They had shackles. Shackles aren’t rope. They aren’t multi-purpose. They have a singular purpose, and that was my final lamenting thought as the tunnel of darkness closed in on me. My head got weak, and fell into the mud. The last sensation I can remember is the taste of blood and dirt in total, silent darkness.\n\nWhen I awoke, I couldn’t move. I was in a pile of my brethren. The weight on me was heavier than anything I can remember. Every damn breath felt like Ebben being rolled over again and again. I was the only one awake. Everyone else pressed against me was warm but the one on my back. He felt cold, and his body never moved. He had voided himself onto me during the night when he passed, but it had likely been long before I awoken, as not only was he, but also his soil cold upon me. Someone shifted as they came out of their own unconscious, and I slid a bit lower. After what was probably 3 hours, I finally could take a breath. The second my face pressed against the wood of the cart I was in, I took an excruciating deep breath of air and passed out.\n\nThe next time I woke up I went from a dark and dreamy haze to the most shockingly violent wave of pure cold crashed over me and I was suddenly awake in water. My lungs were so tired. Every breath felt like fire in the night, and now all I wanted to do was breath. Gods, all I wanted was air. All I could see was a blurry light, but that was all I had so I swam to it. There are moments that humble dwarves. Drowning in what is waist high water for everyone else is the event that does it more than you’d think for our people. All I could hear was them laughing when my ears breached the water, and the stifling quiet of water that felt like it came from a glacier. I could sink to the bottom and with a slight jump I could break the water and take a quick breath. I didn’t think to exhale on the way up the first time, so when I broke the surface all I could get was a mouthful of water when I tried to breathe in and went under after having fully exhaled. My body convulsed to get back to the surface, and my leg happened to kick hard enough against the bottom to get me into the realm of air. My broken ribs rake across each other as I took in that breathe and went back under. Breathing was pain, not breathing was pain, the water was painfully cold. Everything was pain. That’s all there was. In every incarnation.\n\nI felt one grab a handful of my hair, all the way down to the scalp and then yank. Most of the hair held, quite a bit didn’t. Fine shale sheered along my right side as he drug me onto the bank, then I watched as they grabbed another of my men from the wagon and heave him into the five foot deep center of the water that did indeed seem to flow down from the glacier atop the Marrowdwell Mountains above. I couldn’t’ recognize him before they threw him. We were all covered in mud, blood, sweat, vomit, piss and shit before we were cast into the dispelling freeze, and came out surprisingly cleared up of all the filth.\n\nI think it pissed them off when we all rose to our feet, shattered, ragged, but still standing proud. Never have I been more proud of my men. There was no order from me, and no command from the orcs. They simply rose, even then. Because that’s what they do. Hurrik, Lembrun, and Hassal. I’m still trying to find them. They came up to me and unfastened my shackles from behind me and when they saw my eyes begin looking around I saw the knives at my men’s throats. There was nothing in their eyes. Nothing. Fucking good lads. My hands were brought to my front and reshackled to match the others’.\n\nFour shackled dwarves, and nearly three dozen orc bastards in full arms. My men all exchanged the same look.\n\nNot yet.",
        'GARDINER, Mont. – Yellowstone National Park officials assessed widespread damage Tuesday as the park remained closed amid dangerous floods, mudslides and rockslides that have eroded roads, ripped apart bridges and forced evacuations this week.\n\nThe water started to slowly recede Tuesday and one key highway reopened, but the record-level floods left all five entrances to the park closed through at least Wednesday, officials said. And, officials said during an update on Tuesday, the water levels could still change at any time.\n\n“The water is still raging,” said Park Superintendent Cam Sholly, who added that more wet weather was forecast this weekend, which could cause additional flooding.\n\nThe park, which is celebrating its 150th anniversary this year, has seen multiple road and bridge failures, power outages and mudslides, causing the evacuations of more than 10,000 visitors from the park.\n\n“I’ve never seen this, not in my lifetime,” said Austin King, a firefighter and EMT in Gardiner, a town just outside Yellowstone\'s busy northern entrance.\n\nThere were no reports of injuries or deaths as of Tuesday afternoon, but floodwaters swept away a number of homes, bridges and other structures. The northern part of the park suffered the worst damage.\n\nThe full scope of the damage was not known Tuesday, leaving it unclear when roads might reopen or when residents in neighboring communities might be able to return.\n\nThe northern loop of Yellowstone National Park was cleared of visitors and park officials were working to evacuate visitors in the southern area of the park Tuesday. Park staff were searching for five back-country groups to ensure they were safely evacuated.\n\nMeanwhile, Montana Gov. Greg Gianforte declared a statewide disaster and said the Montana National Guard rescued 12 people stranded by high waters in Roscoe and Cooke City.\n\nYELLOWSTONE CLOSED:Yellowstone National Park evacuates visitors\n\nMORE:Home swept away as Yellowstone is hit by major floods and mudslides\n\nHighways reopen as floodwaters recede\n\nThe Yellowstone River reached highs of almost 14 feet on Monday, far higher than the record 11.5 feet set more than a century ago, according the the National Weather Service.\n\nAlthough the water was still "extremely" high, a route out of Gardiner, a town of about 900 people, reopened Tuesday afternoon, according to county Commissioner Bill Berg and Sholly.\n\nVisitors could drive out using U.S. Highway 89 and East River Road, but only residents and service vehicles were allowed to drive back inside. While some residents questioned if they could wait a few days before leaving town, Berg and Sholly were adamant: “Get out of Gardiner.”\n\n"We are working with the county and state of Montana to provide necessary support to residents, who are currently without water and power in some areas," Yellowstone officials said Monday.\n\nA smaller highway into the previously isolated Cooke City also reopened to local traffic and emergency services Tuesday afternoon.\n\nYellowstone communities left stranded, without power\n\nThe flooding left some of Yellowstone\'s small gateway communities in southern Montana isolated and without power, leading to evacuations by boat and helicopter.\n\nFloodwaters isolated areas including Silvergate, which is east of the park, and led to evacuations in Livingston. As Stillwater River in south-central Montana flooded, 68 people were stranded at a campground as crews rescued campers by raft.\n\nOfficials in Park County, which encompasses those cities, issued shelter-in-place orders Monday and warned flooding had made drinking water unsafe in many communities. Residents hauled bottled water home from stores and worried about a possible food shortage.\n\nMany in Gardiner have used only bottled water to brush their teeth, wash dishes and prepare food due to a break in the area\'s main water service line. The community was under an advisory to boil water before consumption as of Tuesday afternoon.\n\nThe county said water and air rescues were underway amid evacuations Monday.\n\n"Extensive flooding throughout Park County has washed out bridges, roads, and left communities and homes isolated," Park County said in a statement.\n\nA 10-person bunker was among the buildings that slipped from the riverbank into the water. Only a portion of the building\'s foundation remained Tuesday.\n\nParker Manning, who was visiting from Terre Haute, Indiana, watched the flooding from a cabin in Gardiner. He said he saw trees and a mostly intact house floating in rushing waters.\n\nIn the south-central Montana town of Joliet, Kristan Apodaca cried as she watched floodwaters overtake her grandmother\'s log cabin and the park where her husband proposed.\n\n"I am sixth-generation," she told the Billings Gazette. "This is our home."\n\nKing, the EMT in Gardiner, said the flooding was "damaging for a lot of people."\n\n"Some have lost their houses; others can’t go to work," King said. "People are worried about food shortages already."\n\nYELLOWSTONE NEWS:National park renames mountain named for officer in Native American massacre\n\nWhen will Yellowstone reopen?\n\nYellowstone officials have prohibited visitors from entering the park at any of its five entrances until at least Wednesday.\n\nSholly said the park could remain closed as long as a week, and northern entrances may not reopen at all this summer.\n\nCory Mottice, a National Weather Service meteorologist in Billings, Montana, said reduced rain and cooler temperatures, which could lead to decreased snowmelt, may lessen flooding.\n\nStill, "this is flooding that we’ve just never seen in our lifetimes before," Mottice said.\n\nWhy is Yellowstone flooding?\n\nRecord rainfall combined with rapidly melting snowpack caused the deluge of flooding this week. Scientists pointed to climate change as the culprit behind more intense and frequent weather events.\n\nThe flooding happened as the summer tourist season was ramping up. June is one of the park\'s busiest months.\n\n\'This is serious\': Millions under heat wave warnings as triple-digit temps move east\n\nWhat state is Yellowstone National Park in?\n\nThe world\'s first national park, Yellowstone National Park encompasses nearly 3,500 square miles on top of a volcanic hot spot. The park lies mostly in Wyoming but spreads into Montana and Idaho.\n\nThe park allows visitors "to observe wildlife in an intact ecosystem, explore geothermal areas that contain about half the world’s active geysers, and view geologic wonders like the Grand Canyon of the Yellowstone River," according to the Yellowstone\'s website.\n\nContributing: The Associated Press\n\nContact News Now Reporter Christine Fernando at cfernando@usatoday.com or follow her on Twitter at @christinetfern.\n\nHannah Phillips of The Palm Beach Post reported from Gardiner, Montana.\n\nYOU MAY LIKE:A first in Yellowstone, as man in Tesla proves his ‘ingenuity’',
        "***For All Things Wyoming, Sign-Up For Our Daily Newsletter***\n\nBy Ellen Fike, Cowboy State Daily\n\nAll entrances to Yellowstone National Park were closed Monday and would be through at least Wednesday as heavy rain falling on melting snow caused significant flooding and road damage, park officials announced.\n\nCalling it “extremely hazardous conditions,” authorities evacuated the northern portion of the park due to flooding, rockslides and mudslides caused by record rainfall.\n\nPark officials announced on Monday afternoon that the park would be closed to visitors until at least Wednesday.\n\n“Red Lodge, Montana, is a river right now and we have some flooding on the west of Cody,” Cody Beers, a Wyoming Department of Transportation spokesman, told Cowboy State Daily. “I’ve never heard of Yellowstone closing the entire park to inbound traffic, though.”\n\nFootage from the storms showed roads collapsing into raging rivers and logs piled high on the shores.\n\nThe electricity was out in several areas of the park as of Monday morning.\n\nAll inbound traffic to the park was halted until conditions stabilized and park officials could assess damage to the roads, bridges and other facilities. The visitors turned away included those with lodging and camping reservations.\n\nMany of the park roads could be closed for an extended period of time, according to park officials.\n\n“It’s just been multiple days of rain up in this area, plus we had snow on Memorial Day weekend,” Beers said. “It’s a rare day when we have water levels that reached the bottom of our bridges.”\n\nCowboy State Daily meteorologist Don Day said flood advisories surround Yellowstone because of the “huge amount of snow” coupled with the significant rain.\n\n“I don’t think anyone thought the runoff would be as big as this,” he said.\n\nBeers said WYDOT will continue to monitor the weather events in the northwestern portion of the state and how they might affect roads.\n\nHe recommended anyone traveling toward Yellowstone wait a day or two for officials to figure out how long certain roads will be closed and whether the rainfall in the area will continue.\n\n“Be careful out there, because Mother Nature’s fury is showing itself right now,” Beers said.\n\nYellowstone superintendent Cam Sholly said on Monday that staff’s first priority was to evacuate the northern section of the park, where there were multiple road and bridge failures, mudslides and other issues.\n\n“The community of Gardiner is currently isolated, and we are working with the county and State of Montana to provide necessary support to residents, who are currently without water and power in some areas,” Sholly said. “Due to predictions of higher flood levels in areas of the park’s southern loop, in addition to concerns with water and wastewater systems, we will begin to move visitors in the southern loop out of the park later today in coordination with our in-park business partners.”\n\nSholly said it was likely the northern loop of the park would be closed for a substantial amount of time.\n\n“I appreciate the efforts of the Yellowstone team and partners to safely evacuate areas of the park and of our gateway community partners who are helping us through this major event,” he said. “We appreciate the support offered by the Department of Interior, National Park Service and the Montana and Wyoming governors.”\n\n***For All Things Wyoming, Sign-Up For Our Daily Newsletter***",
        'Rangers closed all entrances to Yellowstone National Park on Monday because of flooding, rockslides and "extremely hazardous conditions" inside the massive reserve, officials said.\n\nPower outages have also plagued the 2.2 million-acre park, which includes portions of Wyoming, Montana and Idaho, officials said.\n\nIn the 24 hours before mid-day Monday, more than two inches of rain fell on several cities near Yellowstone, including Quadrant, Wyoming (2.68 inches); West Yellowstone, Montana (2.11 inches); Soda Butte, Montana (2.82 inches); and Gardiner, Montana (1.29 inches).\n\nHigh water levels in the Gardner River alongside the North Entrance Road of Yellowstone National Park on June 13, 2022. NPS\n\n"With additional rainfall forecasted, the park does not want large numbers of day-use visitors stranded," according to a Yellowstone statement.\n\nPark superintendent Cam Sholly said in a statement that efforts are underway to "evacuate the northern section" of Yellowstone because of "multiple road and bridge failures, mudslides and other issues."\n\nSholly added that much of Gardiner, Montana, on Monday was "isolated" and that officials "are working with the county and State of Montana to provide necessary support to residents, who are currently without water and power in some areas."\n\nThere\'s more rain in the forecast in the Yellowstone area before it dries out on Wednesday.\n\nFlood watches are in effect for parts of Washington, northern Idaho and northwest Montana. Excessive rainfall in the region could lead to flooding of creeks, streams and other low-lying locations.\n\nUnchecked climate change has been leading to more examples of radical weather, including an increase in floods.',
        '(CNN) — Yellowstone National Park announced Monday all entrances are temporarily closed due to "extremely hazardous conditions" caused by "unprecedented" rainfall and flooding.\n\n"Effective immediately, all entrances to Yellowstone National Park are temporarily CLOSED due to substantial flooding, rockslides and mudslides on roadways from recent unprecedented amounts of rainfall and flooding," the park said in a Facebook post\n\nNo inbound traffic is allowed "until the conditions stabilize and the park can assess damage to roads and bridges," the post added. The North, Northeast, West, South and East entrances are all closed.\n\nThe park posted a situation update on its website Monday afternoon saying that no inbound visitor traffic would be allowed on Tuesday or Wednesday "at a minimum."\n\n"Our first priority has been to evacuate the northern section of the park where we have multiple road and bridge failures, mudslides and other issues," superintendent Cam Sholly said in a statement.\n\nVisitors will be evacuated from the southern loop of the park starting later on Monday due to anticipated higher flood levels and concerns with water and wastewater systems, the statement said.\n\nThe northern loop is likely to be closed "for a substantial amount of time," and the park\'s reopening will be determined after flood waters recede and damage is assessed.\n\nPark staff is working with the county and state to provide support to residents of Gardiner, Montana, a town just north of the park that is currently isolated by the dangerous conditions, the statement said.\n\nMultiple portions of the park are also experiencing power outages, the park said in its postings.\n\n"With additional rainfall forecasted, the park does not want large numbers of day-use visitors stranded in the park," the park said on Facebook and its website.\n\nAccording to National Park Service data from 2014 to 2018, more than 780,000 people typically visit the park in June.\n\n"The river has never been this high before by my house," said Elizabeth Aluck, a resident of Gardiner, which serves as a gateway for visitors.\n\nAs of Monday afternoon, Aluck said she was unable to evacuate because roads and bridges in the area are washed out.\n\nTravelers with plans to visit Yellowstone National Park in the coming weeks should monitor road conditions , the park advised.\n\n\'Things have gotten rougher\'\n\nA family staying at a short-term rental house in Gardiner near the park entrance is now unable to leave their rental cabin due to flooding in the area.\n\nIndiana couple Melissa and Parker Manning told CNN they arrived at their rental on Saturday with their family, and expected to leave Monday morning.\n\n"That\'s not happening any time soon," Parker Manning said. "The water levels were high on Saturday but within the past 10-12 hours things have gotten rougher."\n\nThe couple joined a call with emergency management officials Monday afternoon. Officials on the call suggested local businesses consider food rationing, just in case.\n\nManning said they did go to the grocery store, and everyone was being smart about what to stock up on and not panicking.\n\n"Our way out of town would be north on 89, but those roads are currently all underwater," Manning said.\n\nThe couple has no idea when they will be able to leave the town, but Manning hopes it will be within the next 48 hours.\n\nThe host of their rental was very understandable of the situation, Manning added.\n\nAvoid streams and creeks\n\nEarlier Monday, the park said in a news release roads in the northern portion of the park would be temporarily closed for "an extended period of time" before posting the wider closure.\n\n"Preliminary assessments show multiple sections of road in the park have been washed out between Gardiner and Cooke City, Montana, and multiple bridges may be affected," the release said, adding visitors in the northern portion are being evacuated.\n\nThe National Weather Service issued a flood warning for the park on Monday and advised campers and hikers to avoid streams and creeks.\n\n"Turn around, don\'t drown when encountering flooded roads," the NWS cautioned those in vehicles.\n\nMammoth, Osprey Falls, Indian Creek Campground and Lava Creek Campgrounds, all located in the park, are locations expected to experience floods, the weather service said.\n\nThe Yellowstone River reached record-high levels Monday in the Montana towns of Corwin Springs and Livingston.\n\nAt Corwin Springs, the river rose more than 5 feet on Monday morning, according to the National Oceanic and Atmospheric Administration river gauge data. The gauge reading was 13.85 feet on Monday afternoon, surpassing the historical high crest of 11.5 feet from 1918.\n\nThe river gauge reading at Livingston was a record 10.9 feet.',
        'A road map of Yellowstone National Park indicating points of interest, park road closures due to flooding and improvement projects, and areas where flood impacts occurred. NPS\n\nJune 18, 2022\n\nContact: Morgan Warthin, (307) 344-2015\n\nView more up-to-date information about flood recovery efforts, park operations, the new interim visitor entry system, and FAQ\'s about planning a visit to Yellowstone at go.nps.gov/YELLflood .\n\n\n\nUPDATE: June 18\n\nYellowstone National Park’s south loop will reopen to the public on June 22\n\nVisitors traveling to park in coming weeks must stay informed about the new interim visitor entry system\n\n\n\nAt 8 a.m. Wednesday, June 22, Yellowstone National Park will begin allowing visitors to access the south loop of the park. The south loop is accessed from the East (Cody), West (West Yellowstone), and South (Grand Teton/Jackson) entrances. Accessible areas include Madison, Old Faithful, Grant Village, Lake Village, Canyon Village and Norris. As part of reopening planning, park staff have engaged over 1,000 business owners, park partners, commercial operators and residents in surrounding gateway communities to determine how to manage summer visitation while the north loop remains closed due to flood damage.\n\n\n\nTo balance the demand for visitor access, park resource protection and economic interests of the communities, the park will institute an interim visitor access plan. The interim plan, referred to as the Alternating License Plate System (ALPS), was suggested as a solution by gateway communities during major public engagement with the park this past week. Park managers and partners have agreed this system is the best interim solution to ensuring the south loop does not become overwhelmed by visitors. The National Park Service will actively monitor the license plate system and is concurrently building a new reservation system that will be ready for implementation if needed.\n\nAlternating License Plate System (ALPS)\n\nPublic vehicle entry will be allowed based on whether the last numerical digit on a license plate is odd or even.\n\nEntrance will be granted based on odd/even days on the calendar. Odd-numbered last digits on license plates can enter on odd days of the month. Even-numbered last digits (including zero) on license plates can enter on even days of the month.\n\nPersonalized plates (all letters, for example "YLWSTNE") will fall into the “odd” category for entrance purposes.\n\nPlates with a mix of letters and numbers but that end with a letter (for example "YELL4EVR") will still use the last numerical digit on the plate to determine entrance days.\n\nEntrance station staff will turn away vehicles attempting to enter the park when the odd/even numerical digits do not correspond to the odd/even calendar date for entrance.\n\nExceptions\n\nCurrent commercial use operators with active commercial use permits will be permitted to enter regardless of license plate number. This includes commercial tours and stock groups.\n\nVisitors with proof of overnight reservations in the park will be permitted to enter regardless of license plate number. This includes hotels, campgrounds, and backcountry reservations.\n\nCommercial motorcoaches will be permitted to enter regardless of license plate number.\n\nMotorcycle groups may enter on even dates only.\n\nEssential services like mail and deliver, employees and contractors may enter regardless of license plate number.\n\nThe interim license plate system will ensure that visitors have access to the park during this period of high demand. Park managers and staff will monitor the license plate system and impacts on resources, infrastructure, operations, and staffing, and may adjust or implement a reservation or timed entry system, if necessary, after three to four weeks.\n\n\n\n“Less than six days ago, Yellowstone National Park was hit with devastating floods,” said Superintendent Cam Sholly. “Thanks to the tremendous efforts of our teams and partners, we are prepared to reopen the south loop of Yellowstone. It is impossible to reopen only one loop in the summer without implementing some type of system to manage visitation. My thanks to our gateway partners and others for helping us work out an acceptable temporary solution for the south loop while we continue our efforts to reopen the north loop. As we go through the reopening process, we will monitor the system’s effectiveness and work together to make adjustments that may be necessary. We will also reopen new sections of the park as repairs continue to be made. It is critical for visitors to stay informed about this interim system as we evaluate its effectiveness. They should plan ahead and be patient with us as we are still managing significant recovery while moving into this operational phase.”\n\nOPEN in the south loop as of June 22\n\nCLOSED until further notice in the south loop\n\n\n\nVisit Operating Hours and Seasons for details about what is OPEN and CLOSED throughout the park.\n\n\n\nYellowstone staff are working to determine what other potential sections of the park may be reopened prior to roads closing Nov. 1. Decisions will depend on extent of damage and the ability of park managers to safely open additional sections as the year progresses. Park managers are evaluating plans to reopen roads connecting Canyon Village, Tower Junction, Mammoth Hot Springs and Norris; however, this will not happen initially. The park is also working to reconnect Mammoth Hot Springs to Gardiner and Cooke City/Silver Gate as soon as possible with temporary solutions, while long-term reconstruction is planned.\n\n\n\nVisitors should continue to monitor the park website and for additional updates.\n\nUPDATE: June 17\n\nYellowstone National Park continues efforts to recover from historic flooding; Limited reopening highly possible next week on park’s south loop with certain visitor entrance modifications\n\nVisitors traveling to park in coming weeks must stay informed about current situation, changes in visitor entry requirements, and road and weather conditions\n\nKey Information\n\nYellowstone continues major flood recovery efforts in the north while repairing damage and preparing for reopening in the south.\n\nAll five park entrances remain closed temporarily, however, West, South, and East entrances are targeted for reopening as early as next week.\n\nWater levels remain high but have gone down substantially over the past 24 hours.\n\nWeather patterns and potential for additional flooding events are being monitored with the assistance of NOAA and USGS personnel.\n\nThe National Park Service (NPS) has engaged over 1,000 partners in surrounding gateway communities, counties and states in the past 96 hours to brief on flood damage impacts and collaborate on reopening strategies.\n\nTeams are in the park assessing damage and assisting with short and long-term recovery planning.\n\nNPS is analyzing the carrying capacity of the south loop and working closely with partners to develop appropriate visitor management actions to safely accommodate visitors within that portion of the park.\n\nNPS is working to determine what other potential sections of the park may be reopened for the season. Decisions will depend on extent of damage and the ability of the NPS to safely open additional sections as the year progresses.\n\nAlthough access to Yellowstone National Park will be less than normal until further notice, there are still incredible opportunities for recreation, wildlife viewing, and great experiences in the park’s gateway communities (Gardiner, Montana; Silver Gate and Cooke City, Montana; West Yellowstone, Montana; Cody, Wyoming; and Jackson, Wyoming), as well as surrounding areas in the Greater Yellowstone Ecosystem. View a list of nearby cities, parks, public lands, and museums to help plan your visit.\n\nVisitors who have trips planned should continue to check the Yellowstone website, local chambers of commerce, and social media for updates to get the most up-to-date information.\n\nBecause initial damage assessments are ongoing, the NPS does not yet have an estimate on when Yellowstone will fully reopen nor are preliminary costs for repairs and recovery available.\n\nTo date, there have been no public or employee injuries reported due to the flood.\n\nMessage from the Superintendent\n\n“We have made tremendous progress in a very short amount of time but have long way to go,” said Superintendent Cam Sholly. “All emergency and life safety objectives within the park have been accomplished or stabilized within the first 96 hours of the flood event, without major injury or death. We have an aggressive plan for recovery in the north and resumption of operations in the south. We appreciate the tremendous support from National Park Service and Department of the Interior leadership, in addition to our surrounding Congressional delegations, governors, counties, communities, and other partners. This first 96 hours has been critical to be able to focus on our life safety objectives and stabilizing emergency conditions while preparing plans for recovery.”\n\nObjectives\n\nNorth Loop (Primary Flood Recovery Zone)\n\nEnsure safety of employees, visitors, community residents and partners Restore and maintain basic services including, power, water, and wastewater Restore temporary and permanent access between Yellowstone and Gardiner Restore temporary and permanent road access through the Northeast Entrance Road Restore access to the Yellowstone backcountry Engage communities, local, state, and federal stakeholders, and media\n\nSouth Loop (Operations Zone)\n\nEnsure safety of employees, visitors, and partners Repair damaged sections of roads and infrastructure in preparation of limited south loop reopening; inspect critical infrastructure (bridges, wastewater systems) Conduct extensive community outreach with gateways to identify and determine visitor use management actions necessary to host visitation on the south loop Develop reservation/timed entry system options for implementation with NPS visitor use experts Ensure internal business partners are prepared for visitor operations in the south loop Develop appropriate communications strategy on opening of south loop\n\nGeneral Updates\n\nNorth Loop (Flood Recovery Zone)\n\nAll employees continue to be safe and accounted for. Additional support programs are being established for employees impacted by the flood event.\n\nEmployees who lost housing have been relocated to new housing units.\n\nThanks to Montana Department of Transportation and Park County Montana, Highway 89 north from Gardiner was reopened on Wednesday to facilitate the exit of stranded visitors.\n\nFood and water supplies to Gardiner have resumed.\n\nPower has been restored to Yellowstone with the assistance of Northwestern Energy.\n\nDestroyed wastewater lines in Mammoth have been temporarily repaired with permanent repair options being planned.\n\nExtensive efforts have been made to improve the Old Gardiner Road to help facilitate connection of essential services and personnel from Mammoth to Gardiner. Park staff are assessing what level of improvements are needed to facilitate higher traffic levels.\n\nPlans and assessments of reconstruction of new the permanent road between Mammoth and Gardiner are underway. The new road will likely not be reconstructed in the same corridor as the previous road.\n\nTeams from Federal Highways and other agencies are in the park assisting with damage assessments and assisting the park in developing cost estimates and timeline for repair.\n\nAerial 3D imaging flights have been conducted on the Northeast Entrance Road to Cooke City to help with damage assessments.\n\nExtensive assessments are occurring of damage to trails, bridges and infrastructure within the Yellowstone backcountry.\n\nMud and rockslides have been cleared from Dunraven Pass and other sections of the northern and southern loops.\n\nSouth Loop (Operations Zone)\n\nThe park intends to reopen the south loop sometime next week with new visitor management measures instituted (more information will be released soon). Reopening is dependent on damaged infrastructure being repaired.\n\nOn Thursday morning, June 16, a section of damaged road collapsed south of Canyon Village. Crews will begin major repairs on Friday, June 17 with an intent to complete repairs by Monday, June 20. This repair must be completed prior to opening the south loop.\n\nPark staff are inspecting all bridges and roadways for damage on the south loop to facilitate safe travel.\n\nTo ensure the southern loop infrastructure is not overwhelmed, the park is working with gateway communities and NPS visitor use experts to determine the best way to facilitate access.\n\nKnown Damage and Issues\n\nAerial assessments conducted Monday, June 13 by Yellowstone National Park show major damage to several park roads including: North Entrance (Gardiner, Montana) to Mammoth Hot Springs: road washed out in multiple places, significant rockslide in Gardner Canyon Tower Junction to Northeast Entrance: segment of road washed out near Soda Butte Picnic Area, mudslides, downed trees Tower-Roosevelt to Canyon Junction (Dunraven Pass): mudslide on road Canyon Junction to Fishing Bridge: Segment of road just south of Canyon Junction is compromised and collapsed overnight on June 15-16. Repairs are underway.\n\nMany sections of road in these areas are completely gone and will require substantial time and effort to reconstruct or relocate and build.\n\nIn some cases, roads may not be able to be reconstructed in place and will need to be relocated.\n\nThe NPS will make every effort to repair these roads as soon as possible.\n\nStay Informed\n\nVisitors planning to travel to Yellowstone in the upcoming weeks should stay informed about the current situation and pay close attention to the status of road and weather conditions.\n\nStay informed about up-to-date road conditions in Yellowstone: Visit Park Roads. To receive Yellowstone road alerts on your mobile phone, text “82190” to 888-777 (an automatic text reply will confirm receipt and provide instructions). Call (307) 344-2117 for a recorded message.\n\nFind updates about flood recovery efforts and park operations at go.nps.gov/YELLflood.\n\nFind new photos Flickr of this extremely hazardous situation in Yellowstone National Park.\n\nphotos Flickr of this extremely hazardous situation in Yellowstone National Park. Yellowstone will continue to communicate about this hazardous situation as more information is available.\n\nUPDATE: June 14\n\nNorthern portion of Yellowstone National Park likely to remain closed for a substantial length of time due to severely damaged, impacted infrastructure\n\nVisitors traveling to park in coming weeks must stay informed about current situation, road and weather conditions\n\nUpdates\n\nAerial assessments conducted Monday, June 13, by Yellowstone National Park show major damage to multiple sections of road between the North Entrance (Gardiner, Montana), Mammoth Hot Springs, Lamar Valley and Cooke City, Montana, near the Northeast Entrance.\n\nMany sections of road in these areas are completely gone and will require substantial time and effort to reconstruct.\n\nThe National Park Service will make every effort to repair these roads as soon as possible; however, it is probable that road sections in northern Yellowstone will not reopen this season due to the time required for repairs.\n\nTo prevent visitors from being stranded in the park if conditions worsen, the park in coordination with Yellowstone National Park Lodges made the decision to have all visitors move out of overnight accommodations (lodging and campgrounds) and exit the park.\n\nAll entrances to Yellowstone National Park remain temporarily CLOSED while the park waits for flood waters to recede and can conduct evaluations on roads, bridges and wastewater treatment facilities to ensure visitor and employee safety.\n\nThere will be no inbound visitor traffic at any of the five entrances into the park, including visitors with lodging and camping reservations, until conditions improve and park infrastructure is evaluated.\n\nThe park’s southern loop appears to be less impacted than the northern roads and teams will assess damage to determine when opening of the southern loop is feasible. This closure will extend minimally through next weekend (June 19).\n\nDue to the northern loop being unavailable for visitors, the park is analyzing how many visitors can safely visit the southern loop once it’s safe to reopen. This will likely mean implementation of some type of temporary reservation system to prevent gridlock and reduce impacts on park infrastructure.\n\nAt this time, there are no known injuries nor deaths to have occurred in the park as a result of the unprecedented flooding.\n\nEffective immediately, Yellowstone’s backcountry is temporarily closed while crews assist campers (five known groups in the northern range) and assess damage to backcountry campsites, trails and bridges.\n\nThe National Park Service, surrounding counties and states of Montana and Wyoming are working with the park’s gateway communities to evaluate flooding impacts and provide immediate support to residents and visitors.\n\nWater levels are expected to recede today in the afternoon; however, additional flood events are possible through this weekend.\n\nKnown damage and issues\n\nKnown damage (at this time) to some park roads includes: North Entrance (Gardiner, Montana) to Mammoth Hot Springs: road washed out in multiple places, significant rockslide at Gardner Canyon Tower Junction to Northeast Entrance: segment of road washed out near Soda Butte Picnic Area, mudslides, downed trees Tower-Roosevelt to Canyon Junction (Dunraven Pass): mudslide on road Canyon Junction to Fishing Bridge: Segment of road just south of Canyon Junction potentially compromised and closed for evaluation\n\nThe power continues to be out in multiple locations in the park.\n\nWater and wastewater systems at Canyon Village and Mammoth Hot Springs are being impacted by flooding conditions and are being monitored.\n\nStay informed\n\nVisitors planning to travel to Yellowstone in the upcoming weeks should stay informed about the current situation and pay close attention to the status of road and weather conditions.\n\nStay informed about up-to-date road conditions in Yellowstone: Visit Park Roads. To receive Yellowstone road alerts on your mobile phone, text “82190” to 888-777 (an automatic text reply will confirm receipt and provide instructions). Call (307) 344-2117 for a recorded message.\n\nFind photos on Flickr of this extremely hazardous situation in Yellowstone National Park.\n\nYellowstone will continue to communicate about this hazardous situation as more information is available.\n\nUPDATE: June 13, 2022 at 4:32 p.m.\n\nPlease find a video just posted to Flickr that underscores the severity of the situation in Yellowstone National Park. The park’s helicopter manager recorded the video from the park’s helicopter of the Gardner River and portions of the road between Mammoth Hot Springs and the North Entrance through the Gardner Canyon. Segments of the paved road are eroded and washed out in several places due to high water levels.\n\nYellowstone will continue to communicate about this hazardous situation as more information is available.\n\nUPDATE: June 13, 2022 at 2:15 p.m.\n\nAll entrances to Yellowstone National Park CLOSED temporarily due to heavy flooding, rockslides, extremely hazardous conditions\n\nStay informed about road status and weather conditions\n\nEffective immediately, there will be no inbound visitor traffic at any of the five entrances into Yellowstone National Park on Tuesday, June 14, and Wednesday, June 15, at a minimum.\n\nStay informed about up-to-date road conditions in Yellowstone: Visit Park Roads. To receive Yellowstone road alerts on your mobile phone, text “82190” to 888-777 (an automatic text reply will confirm receipt and provide instructions). Call (307) 344-2117 for a recorded message.\n\nFind photos and videos on Flickr of this extremely hazardous situation in Yellowstone National Park.\n\nYellowstone will continue to communicate about this hazardous situation as more information is available.\n\nUPDATE: June 13, 2022 at 1:16 p.m.\n\nStatement from superintendent Cam Sholly about all entrances in Yellowstone National Park CLOSED temporarily due to heavy flooding, rockslides, extremely hazardous conditions\n\n“Due to record flooding events in the park and more precipitation in the forecast, we have made the decision to close Yellowstone to all inbound visitation," said Superintendent Cam Sholly. "Our first priority has been to evacuate the northern section of the park where we have multiple road and bridge failures, mudslides and other issues. The community of Gardiner is currently isolated, and we are working with the county and State of Montana to provide necessary support to residents, who are currently without water and power in some areas. Due to predictions of higher flood levels in areas of the park’s southern loop, in addition to concerns with water and wastewater systems, we will begin to move visitors in the southern loop out of the park later today in coordination with our in-park business partners. We will not know timing of the park’s reopening until flood waters subside and we\'re able to assess the damage throughout the park. It is likely that the northern loop will be closed for a substantial amount of time. I appreciate the efforts of the Yellowstone team and partners to safely evacuate areas of the park and of our gateway community partners who are helping us through this major event. We appreciate the support offered by the Department of the Interior, National Park Service and the Montana and Wyoming governors."\n\n\n\nYellowstone will continue to communicate about this hazardous situation as more information is available.\n\n\n\n\n\nUPDATE: June 13, 2022 at 11:10 a.m.\n\nAll entrances to Yellowstone National Park CLOSED temporarily due to heavy flooding, rockslides, extremely hazardous conditions\n\nStay informed about road status and weather conditions\n\nEffective immediately, all entrances to Yellowstone National Park are temporarily CLOSED due to substantial flooding, rockslides and mudslides on roadways from recent unprecedented amounts of rainfall and flooding.\n\nEffective immediately, no inbound visitor traffic will be allowed into the park until conditions stabilize and the park can assess damage to roads and bridges and other facilities. This includes visitors with lodging and camping reservations.\n\nClosed entrances include: North Northeast West South East\n\nThe power is out in multiple locations in the park.\n\nVisitors planning on coming to Yellowstone in the upcoming weeks should pay close attention to the status of road conditions.\n\nMany park roads may remain closed for an extended period of time.\n\nPreliminary assessments show multiple sections of roads throughout the park have been either washed out or covered in mud or rocks, and multiple bridges may be affected.\n\nMultiple roads in the southern portion of the park are also on the verge of being flooded, further restricting access.\n\nWith additional rainfall forecasted, the park does not want large numbers of day-use visitors stranded in the park.\n\nStrains on wastewater and water treatment facilities could become a factor and the park is taking precautions to ensure facilities are not failing.\n\nThe National Park Service, surrounding counties and state of Montana and Wyoming will work with the gateway communities to evaluate flooding impacts and provide support to residents.\n\nRainfall is expected to continue for the next several days. Flood levels measured on the Yellowstone River are beyond record levels.\n\nStay informed about up-to-date road conditions in Yellowstone: Visit Park Roads. To receive Yellowstone road alerts on your mobile phone, text “82190” to 888-777 (an automatic text reply will confirm receipt and provide instructions). Call (307) 344-2117 for a recorded message.\n\nFind photos on Flickr of this extremely hazardous situation in Yellowstone National Park.\n\nYellowstone will continue to communicate about this hazardous situation as more information is available.\n\nJune 13, 2022 at 9:21 a.m.\n\nRoads in northern portion of Yellowstone National Park CLOSE temporarily due to heavy flooding, rockslides, extremely hazardous conditions\n\nStay informed about road status and weather conditions',
        "As summer tourist season ramps up, record-breaking floods in Yellowstone National Park and dangerous mudslides have swept away homes, ripped apart bridges and forced evacuations of surrounding communities isolated without power.\n\nOn Tuesday, a video showed a home in Gardiner, Montana, collapsing into the murky waters of Yellowstone River.\n\nYellowstone River reached highs not seen in our lifetimes, at almost 14 feet on Monday. The previous record of 11.5 feet occurred more than a century ago, according to the U.S. Geological Survey.\n\nYellowstone National Park covers nearly 3,500 square miles in Wyoming, Montana and Idaho. It sits atop a volcanic hot spot with about half the world’s active geysers.\n\nNearby Park County said water and air rescues were underway Monday and issued shelter-in-place orders.\n\nMore:Yellowstone National Park closes entrances, evacuates visitors amid 'unprecedented' rainfall, flooding\n\nClimate change is responsible for more intense and frequent weather events, scientists have said.\n\nRainfall is expected for the next several days and all five entrances to the park were closed, officials said Monday. There were no immediate reports of injuries.\n\nCamille Fine is a trending visual producer on USA TODAY's NOW team. She loves to make pizza, photograph friends and spoil her loving cat Pearl.",
        "Record flooding and rockslides following a burst of heavy rains prompted the rare closure on Monday of all five entrances to Yellowstone national park at the start of the summer tourist season, the park superintendent said.\n\nThe entire park, spanning parts of Wyoming, Montana and Idaho, will remain closed to visitors, including those with lodging and camping reservations, at least through Wednesday, as officials assess damage to roads, bridges and other facilities.\n\nCurrent conditions of Yellowstone’s North Entrance Road through the Gardner Canyon between Gardiner, Montana, and Mammoth Hot Springs.\n\n\n\nWe will continue to communicate about this hazardous situation as more information is available. More info: https://t.co/mymnqGvcVB pic.twitter.com/S5ysi4wf8a — Yellowstone National Park (@YellowstoneNPS) June 13, 2022\n\nAll five park entrances were closed to inbound traffic for the first summer since a series of devastating wildfires in 1988. The National Park Service was working to reach visitors and staff remaining at various locations, especially in the hardest-hit northern flank of Yellowstone, officials said.\n\n“Our first priority has been to evacuate the northern section of the park where we have multiple road and bridge failures, mudslides and other issues,” the park superintendent, Cam Sholly, said in a statement.\n\nGardiner River along the north entrance to Yellowstone national park is high enough to have washed out part of the road. Photograph: AP\n\nThe “gateway” community of Gardiner, Montana, just north of the park’s northern boundary and home to many Yellowstone staff members, has been cut off due to a mudslide in the vicinity, according to the National Park Service.\n\nElectric power was knocked out in multiple areas of the park, and preliminary assessments showed numerous sections of roads throughout Yellowstone either washed away or covered in rocks and mud, with a number of bridges also damaged, the park service said.\n\nVarious roads in the park’s southern tier were on the verge of being flooded, according to the agency.\n\n6/13/22 at 2:15 p.m. UPDATE: Effective immediately, there will be no inbound visitor traffic at any of the five entrances into Yellowstone National Park on Tuesday, June 14, and Wednesday, June 15, at a minimum. Learn more: https://t.co/mymnqGMNN9 https://t.co/Kz2sEbOS7a — Yellowstone National Park (@YellowstoneNPS) June 13, 2022\n\nThe flooding and slides were triggered by days of torrential showers in the park and steady rains across much of the wider region after one of its wettest springs in many years. The park service characterized the levels of rainfall and flooding sweeping the park as unprecedented.\n\nA sudden spike in summer temperatures during the past three days also has hastened melting and runoff of snow accumulated in the park’s higher elevations from late-winter storms.\n\nThe heavy rains and rapid runoff of snow melt converged to create treacherous conditions in the park just two weeks after the traditional Memorial Day holiday weekend kickoff of the US summer tourist season, which accounts for the bulk of Yellowstone’s annual 4 million visitors.",
        "Portland, Oregon 2022-06-13 19:27:22 –\n\nHelena, Mon. (NEXSTAR) —On Monday, a deluge wiped out at least one bridge, washed away roads, and caused a landslide in Yellowstone National Park.\n\nAuthorities closed all five entrances, began evacuating visitors from the northern part of the park, and warned that the park was temporarily closed due to a “very dangerous situation.”\n\nPark officials said roads to and from the Gardiner community in Montana, just north of the park, were impassable. Kam Sholy said the loop on the north side of the park could be closed for quite some time.\n\nPark officials planned to evacuate visitors from the park’s South Loop on Monday afternoon. This is because more rain is expected.\n\nHigher levels of flood forecasts prompted the decision, along with concerns about water and wastewater, according to Shory.\n\nThis photo, provided by the National Park Service, shows the high water levels of the Gardner River along the north exit of Yellowstone National Park, Montana, which washed away part of the road on Monday, June 13, 2022. (National Park Service via AP)\n\nThis photo, provided by the National Park Service, is a large rockslide on the North Entrance Road of Gardner Canyon in Yellowstone National Park, Montana, on Monday, June 13, 2022. All entrances to Yellowstone National Park were temporarily closed on Monday due to a major flood. Park officials said, rockslides and landslides on the road from recent unprecedented amounts of rainfall and floods. (National Park Service via AP)\n\nThis image from the National Park Service’s webcam is the west entrance to the Yellowstone National Park in Montana on Monday, June 13, 2022. (National Park Service via AP)\n\nThis photo, provided by the National Park Service, is a bridge washed away from floods at Rescue Creek in Yellowstone National Park, Montana, on Monday, June 13, 2022. (National Park Service via AP)\n\nHelicopter video What is shared in the park shows a road that has been completely washed away in some places by the river next door. Park officials said in a statement that “an unprecedented amount of rainfall” caused the flood.\n\nThe park entrance will remain closed until at least Wednesday.\n\n“We don’t know when the park will reopen until the floods subside. We can assess the damage to the entire park,” says Sholly. “Thanks to Yellowstone’s team and partners for their efforts to safely evacuate the park area and for the efforts of the Gateway Community Partners who have helped us through this major event.”\n\nFor updated terms, please visit the following website: Park road Alternatively, send a text message to 888-777 with “82190” to receive a Yellowstone road alert.\n\nThe Associated Press contributed to this report.",
        'A series of “unprecedented” rainstorms have created chaos inside Yellowstone National Park, which closed its gates to visitors on Monday.\n\nThe severe weather caused mudslides, with multiple roads being rendered impassible, and a bridge getting destroyed.\n\n"Our first priority has been to evacuate the northern section of the park where we have multiple road and bridge failures, mudslides and others issues," park superintendent Cam Sholly said in a statement.\n\nGuests already inside the park have been evacuated, and no return date has been announced.\n\nVideo footage released the National Parks Service (NPS) shows the northern entrance road to the park from Gardiner, Montana destroyed as water rushes through the canyon.\n\nCommunities outside of Yellowstone like Gardiner have been isolated, as roads going in and out of town are rendered impassible.\n\nOne family staying in the area in a rental home told CNN they were stranded.\n\n“The water levels were high on Saturday but within the past 10-12 hours things have gotten rougher,” said Parker Manning, who was visiting from Indiana.\n\nWater levels on the Yellowstone River in Corwin Springs, Montana had blown past its previous record high by at least two feet, according to the US Geological Survey. Other rivers in the area were also far above normal levels.\n\nPower is out in some parts of the popular natural park.\n\nIt is unclear when the park could re-open.\n\n“We will not know timing of the park’s reopening until flood waters subside and we’re able to assess the damage throughout the park,” Mr Sholly said in the statement. “It is likely that the northern loop will be closed for a substantial amount of time.”\n\nThe flooding was caused by intense rainfall, coupled with increased snowmelt. The AP reports that the park saw 2.5 inches of rain fall from Saturday to Monday, with areas nearby recording up to four inches.\n\nIn addition, recent warmer temperatures have spurred increased snowmelt in the area, adding even more water to the landscape.\n\nThe park is one of the most popular outdoor recreation spots in the US, receiving more than 800,000 visitors a year on average.\n\nThe climate crisis is expected to bring significant change to the Yellowstone ecosystem, NPS says. In particular, warmer temperatures will likely bring decreased snowpack and a higher chance of wildfires.\n\nClimate change may also bring more severe storms across the country. According to the US Environmental Protection Agency, more rain is now coming during intense rainstorms than in years prior — which could lead to more flash flooding events.\n\nA recent study found that floods in the US would get “flashier” as the climate warms further, leading to quicker, more intense flooding events.',
        "CODY, Wyo., June 13 (Reuters) - Record flooding and rockslides unleashed by an unprecedented burst of heavy rains prompted the rare closure on Monday of all five entrances to Yellowstone National Park at the start of the summer tourist season, the park superintendent said.\n\nThe entire park, spanning parts of Wyoming, Montana and Idaho, will remain closed to visitors, including those with lodging and camping reservations, at least through Wednesday, as officials inspect damage to roads, bridges and other facilities.\n\nThe closures come as Yellowstone was gearing up to celebrate its 150th anniversary year, and as local communities heavily dependent on tourism were counting on a rebound following COVID-19 travel restrictions over the past two summers.\n\nRegister now for FREE unlimited access to Reuters.com Register\n\nAll five park entrances were closed to inbound traffic for the first summer since a series of devastating wildfires in 1988. The National Park Service said it was working to evacuate visitors and staff remaining at various locations, especially in the hardest-hit northern flank of Yellowstone.\n\n\"It is likely that the northern loop will be closed for a substantial amount of time,\" the park superintendent, Cam Sholly, said in a statement.\n\nThe \"gateway\" community of Gardiner, Montana, just beyond the park's northern boundary and home to many of Yellowstone's workers, was cut off by a mudslide to the north and washed-out road surfaces to the south, according to the National Park Service.\n\n1/3 High water levels in the Lamar River erode Yellowstone National Park's Northeast Entrance Road, where the park was closed due to heavy flooding, rockslides, extremely hazardous conditions near Gardiner, Montana, U.S. June 13, 2022. Picture taken June 13, 2022. National Park Service/Handout via REUTERS. Read More\n\nAerial footage released by the Park Service showed large swaths of the winding North Entrance Road between Gardiner and park headquarters in Mammoth Hot Springs, Wyoming, carved away by surging floodwaters along the Gardner River - washouts that will likely take months to fully repair.\n\nPower outages were scattered throughout the park, and preliminary assessments showed numerous roadways across Yellowstone either washed away or covered in rocks and mud, with a number of bridges also damaged, the agency said.\n\nVarious roads in the park's southern region were on the verge of being flooded, with more rain in the forecast.\n\nThe flooding and slides were triggered by days of torrential showers in the park and steady rains across much of the wider Intermountain West following one of the region's wettest springs in many years. The park service characterized the rainfall and floods sweeping the park as unprecedented, with the Yellowstone River topping its banks beyond record levels.\n\nA sudden spike in summer temperatures during the past three days also has hastened melting and runoff of snow accumulated in the park's higher elevations from late-winter storms.\n\nThe heavy rains and rapid runoff of snow melt converged to create treacherous conditions in the park just two weeks after the traditional Memorial Day holiday weekend kickoff of the U.S. summer tourist season, which accounts of the bulk of Yellowstone's annual 4 million visitors.\n\nYellowstone, established as the world's first national park in 1872 and treasured as one of America's top outdoor travel destinations, occupies some 2.2 million acres (890,308 hectares)famed for its geysers, abundant wildlife and spectacular scenery.\n\nRegister now for FREE unlimited access to Reuters.com Register\n\nReporting by Ruffin Prevost in Cody, Wyoming; Writing and additional reporting by Steve Gorman in Los Angeles; Editing by Sandra Maler\n\nOur Standards: The Thomson Reuters Trust Principles.",
        'This is the best tl;dr I could make, original reduced by 67%. (I\'m a bot)\n\nUpdate 11:05 a.m. Officials have closed all inbound traffic into Yellowstone National Park "Due to extremely hazardous conditions from unprecedented amounts of rainfall."\n\nAll entrances to Yellowstone National Park CLOSED temporarily due to heavy flooding, rockslides, extremely hazardous conditions; Stay informed about road status and weather conditions!\n\nEffective immediately, all entrances to Yellowstone National Park are temporarily CLOSED due to substantial flooding, rockslides and mudslides on roadways from recent unprecedented amounts of rainfall and flooding.\n\nEffective immediately, no inbound visitor traffic will be allowed into the park until conditions stabilize and the park can assess damage to roads and bridges and other facilities.\n\nYELLOWSTONE NATIONAL PARK - Roads in the northern portion of Yellowstone National Park were temporarily closed Monday morning due to substantial flooding, rockslides and mudslides on roadways from recent unprecedented amounts of rainfall.\n\nThe National Park Service, Park County, Montana, and state will work with the communities of Gardiner, Silver Gate and Cooke City to evaluate flooding impacts and provide support to residents.\n\nSummary Source | FAQ | Feedback | Top keywords: Park#1 road#2 close#3 Yellowstone#4 Flood#5\n\nPost found in /r/worldnews.\n\nNOTICE: This thread is for discussing the submission topic. Please do not discuss the concept of the autotldr bot here.',
        "Visitors to Yellowstone National Park were seen narrowly escaping with their lives after a rockslide began falling on their vehicle amid devastating floods.\n\nThe drama took place when heavy rain fell on northern Wyoming and southern Montana on Sunday, causing record-breaking flooding into Monday and Tuesday.\n\nA video shared to social media by Twitter user Anne Leopold showed one SUV approaching a turn on a road while rock slabs fell from the slope above.\n\nPieces of rock smashed and broke around the SUV, which Ms Leopold said had occupants inside who “seemed okay” despite some rock appearing to strike the vehicle.\n\nBoth the SUV and the Twitter user, who was filming the drive out of Yellowstone, had been heading towards the northern gate by Gardiner -- a Montana town badly affected by the water levels.\n\nOnly one road out of the tourist town remains open, according to Mike Skelton, the president of the local chamber of commerce.\n\nHe told KULR-8 news on Wednesday: “As you know being down here, there were several thousand visitors that were stuck here in Gardiner and couldn’t get out through the park, couldn’t get out through Highway 89”.\n\nRoughly 10,000 people have been evacuated from Yellowstone since Sunday as floodwaters submerged critical bits of infrastructure and popular trails. Only a group of campers remains, officials said.\n\nResidents of neighbouring towns have meanwhile taken-in thousands of visitors and power supplies continue to be affected, as does drinking water. A number of homes and buildings have also been damaged but no fatalities have been reported.\n\nSome 87 people have been rescued by the Montana National Guard following the declaration of a disaster by state governor Greg Gianforte, the agency said on Wednesday.\n\nWater levels in some parts are falling but according to the National Weather Service in Billings, Montana, the flood remains at the highest warning level, of 16ft.\n\nAdditional reporting by the Associated Press.",
        "There were no casualties as a result of this rockfall event. However, the island has been closed to allow for a full examination of the site and to clear debris.",
        "GANGTOK: Sikkim woke up to news of multiple landslides and road blockages across the state after a night of heavy downpour.\n\nIn visuals shared across social media, a luxury taxi SUV was seen stuck in slush from the 20th Mile landslide on NH 10. No passengers or drivers were harmed in the landslide, the Pakyong district administration said.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nThe landslide at 20th Mile between the riverside towns of Rangpo and Singtam in East Sikkim has been a common cause of concern for decades now.\n\nPakyong district administration informed that for safety, the traffic has been diverted towards Duga village route from Singtam to reach Rangpo, even as work on clearing the landslide is on while allowing partial movement of the vehicles.\n\nClose to the 20th Mile landslide, the Mining bridge connecting East with South Sikkim near Mamring town is also blocked after an uprooted tree fell on the bridge.\n\nAlso Read | In pics: Sikkim celebrates Saga Dawa at Tsuklakhang Palace\n\nTrending Stories\n\n\n\n\n\n\n\n\n\n",
        "Guwahati: Assam continued to reel under heavy rain for the third consecutive day with two more deaths reported due to landslides on Wednesday, and the flood situation taking a turn for the worse, officials said.\n\nTwo children were crushed to death when a landslide led to a house collapse in Azad Nagar area of Goalpara district, raising the toll due to floods and landslides in Assam this year rose to 44, they said.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nThe two deceased children were identified as 11-year-old Hussain Ali and eight-year-old Asma Khatoon.\n\nIn Guwahati, three persons suffered injuries in Noonmati area as multiple landslides were reported across the city during the day.\n\nIn several areas, including Joypur in Kharguli area, Bonda Colony, South Sarania, Amayapur in Geetanagar and 12 Mile, debris pileup led to road blockades.\n\nWork to clear the road leading to Nizarapar, where stands Bharat Ratna Dr Bhupen Hazarika’s residence, was underway, the officials stated.\n\nTwo children were crushed to death when a landslide led to a house collapse in Azad Nagar area of Goalpara district\n\nTorrential rain has also damaged two electric poles near Raj Bhavan here.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nAt least 18 districts in the state are still experiencing heavy showers, with inundation of fresh areas reported from Kamrup Metro, Kamrup, Nalbari and Barpeta.\n\nNearly 75,000 people have been affected by floods in the 18 districts.\n\nWater level in river Brahmaputra and its tributaries are on the rise, while river Manas was flowing above the danger mark in some places.\n\nLife has come to a standstill in Guwahati due to continued waterlogging in most parts for the third consecutive day with areas such as Anil Nagar, Nabin Nagae, Zoo Road, Six Mile, Noonmati, Bhootnath, Maligaon figuring among the worst-affected.\n\nThe administration has urged people not to venture out of their homes unless necessary.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nDeputy commissioners in Lower Assam districts have also issued directives to educational institutions to suspend classes till Saturday as the Regional Meteorological Centre’s warned of ‘heavy to very heavy and extremely heavy rainfall in Assam and Meghalaya’.\n\nThe centre has issued ‘red alert’ for Assam and Meghalaya from Tuesday to Thursday and ‘orange alert’ on Friday and Saturday.\n\nDistrict authorities have opened seven camps and nine relief distribution centres for assisting locals. A total of 1,224 people have taken shelter in the relief camps 677 alone in Kamrup Metropolitan\n\nAccording to the bulletin issued by Assam State Disaster Management Authority (ASDMA), 13 embankments were breached, 64 roads and a bridge damaged thus far due to heavy rain and flood.\n\nAlso read: Assam: Paper mill unions express relief at Rs 500 crore package\n\nADVERTISEMENT CONTINUE READING BELOW\n\nTrending Stories\n\n\n\n\n\n\n\n\n\n",
        "Gangtok: The Army on Wednesday carried out a rescue mission evacuating eight people buried under debris following a landslide in East Sikkim, officials said.\n\nThe troops of the Black Cat Division swung into action and rescued the people at 17th Mile, they said, adding, that one of them died in hospital.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nAll the evacuees were admitted to the nearest military hospital, the officials said.\n\nAlso read: Sikkim: Three policemen die in road accident in Dzongu\n\nTrending Stories\n\n\n\n\n\n\n\n\n\n",
        "GANGTOK: Arunachal Pradesh resident Kalu Tamang (35), working as a labour in Sikkim, died in a landslide on Tuesday.\n\nThe landslide occurred in an AG Construction Company Area (mess room) and some labourers got trapped under the landslide debris near 17th Mile on the Gangtok-Nathula Highway.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nTwo other labourers were rescued and referred for medical assistance to STNM Hospital in Gangtok. The Indian Army, Forest officials and the Sherethang police station carried out a rescue operation. However, Tamang, a resident of Bhalukpong in Arunachal Pradesh succumbed to his injuries.\n\nAll labourers have been shifted from effected area to new location, a report from Sikkim Police said.\n\nTrending Stories\n\n\n\n\n\n\n\n\n\n",
        "<< First | < Prev | Next >\n\n***\n\n“So you’re just going to leave, then?” Muqwa shouted at the scattered throngs of priests and clerics. “You cowards! Apostates!”\n\nAvians of every feather hung their heads in shame, but they did not stop walking away. They abandoned their temples and the libraries that grew up the cliff walls and against the tower itself. Most of them carried knapsacks full of books or relics.\n\nPoire and Muqwa were standing in the shadow of the easternmost tower. Ages ago, a huge section of the cliff wall had collapsed here, and the landslides wrapped around the base of the tower. Stone workers cut at the landslide so that the tower seemed to have been chiseled out of the rock itself. Walkways and switchbacks crawled up the cliff, leading to dozens of shrines and homes for the siblings of the Faith and the working clergy.\n\nNow, those siblings were running away. Most of them were avian, but Poire spotted one creature with bulging yellow eyes and green, leathery scales wearing a priest’s walking clothes. Some were rushing. Others tried to move with a quiet dignity, as if they thought they were doing the right thing.\n\n“Where are you going to go, then?” Muqwa shouted at the throngs. He pointed at one red-feathered avian. “Sister Inakah, what are you going to do?”\n\n“Survive,” Inakah said. She was just as old and round as Muqwa. “At least, I hope to. And if you know better, you will leave too, Muqwa.”\n\n“What about the Oracle? What about our order?”\n\n“The Oracle is silent, as always.”\n\nShe was already walking away.\n\n“Brother Osul,” Muqwa called to another avian, who was carrying a bulging knapsack over his shoulder. Silver and other precious metals glinted out of the top. “Are those from the temple? Are you stealing from the gods?”\n\n“Mind your own roost, Muqwa. The gods help those who help themselves.”\n\nMuqwa opened his mouth to argue when Poire tugged at his robes. Reminding him of their purpose.\n\nBut Muqwa was too worked up. His jowls quivered with fury. “You call yourselves faithful. Pwah! Faith would have you here to protect our sacred home from these ransacking cyrans. What if the Savior returns? Where will you be?”\n\nOne or two of the priests stopped long enough to look back at Muqwa, at the temple and the tower. But they would not change their minds. The fire over the Midcity was growing taller. The clouds, blacker.\n\nBrother Osul, on the other hand, was still standing there, his brow feathers cocked high.\n\n“Who’s this?” Osul nodded at Poire.\n\n“A lost fledgling, that’s all.”\n\nOsul stared.\n\nMuqwa put a protective hand out, pushing Poire behind him. “I found him on my way from the garden.”\n\nBut the answer didn’t seem to satisfy Osul. Poire could feel the brother’s gaze lingering on his robes, searching for his face beneath the shadow of his hood. Poire shifted uncomfortably, and even his armor began to stir again somewhere beneath his clothes. He pulled his hood tighter over his face.\n\nThere was a shout behind them. A patrol of cyrans were shoving one of the priests around, pulling a knapsack off her shoulders and dumping its contents onto the ground. She threw up her hands, letting them take it, letting them tear through the books and papers.\n\n“Quickly,” Muqwa whispered to Poire, turning him away from Osul, “let’s get inside before anyone sees you.”\n\nThe easternmost tower, just like the leaning tower, was not how Poire remembered it. Huge stone walls had been constructed around the tower, completely walling in the central pylon. Still, the tower was unmistakable. It soared hundreds of feet above the other buildings.\n\nArches ringed the base of the tower, forming dozens of entrances to the inner monastery.\n\nMuqwa went in first to make sure that it was clear to enter. And then, he beckoned Poire to follow.\n\nInside, the sound of the world changed. Footsteps became shuffling echoes, and every breath sounded like the rumbling earth. Torches and pockets filled with candles lit the dusky atrium. There was a kind of excitement in the priest’s gait as he led them down a set of steep, seastone steps that curved around the center of the tower.\n\n“Right this way. And watch the stairs, they’re sharp.”\n\nThe steps were cramped and worn smooth from centuries of use; Poire’s feet wanted to slip on them. Candles and torches were replaced by gaslights encased in fragile, cloudy glass. A glow ushered in shadows from below.\n\nHere, the lights numbered in the hundreds. A cavernous basement stretched as far as he could see, filled to brimming with the steady warmth of lanterns and the glint of gold. Precious metals had been crafted into symbols of the Faith that hung on the walls and the heavy, stone columns. Color had been worked into the stones so that the polished floor was one enormous mural.\n\nThe smell of natural gas was so rich it was hard to think. All Poire could hear was the hissing of fuel flowing through a network of pipes that crawled across the corners of the floor.\n\nThe ceilings were covered with paintings, flecked and peeling. Each one showed another scene, often of human beings. Sometimes they were tending to plants or feeding animals. This one was a woman whispering into the ear of a bird. Here was a man reaching into the water where a scaled hand rose up to meet his. And directly above Poire was a human sitting cross-legged with his eyes closed, a whole forest of trees growing out of his flesh. Each scene was a new masterpiece.\n\nThey built all this? Burning gas to see and carving with such crude tools?\n\nThere was a gleam of metal from the center of the temple. One of eight pylons, built by humans so long ago. The pylon held up all this stone, just like it once held something else.\n\nBut Muqwa drew his attention to an altar covered with a pale linen cloth. Gold stitching embroidered the cloth in an intricate pattern.\n\n“There it is.” Muqwa bowed at the altar. “The Oracle.”\n\n“It’s a control panel.”\n\n“Hmm?” Muqwa said.\n\nBut how could Poire explain? This waist-high station, this dome, was just an access node. The tower’s manual control, mostly used for troubleshooting. The chrome shell was worn with a patina of scratches, scrapes, and smudges of tarnish where generations of hands had touched the shell in the same spots.\n\nMaybe it still has power, Poire thought.\n\n“It looks like it needs maintenance.”\n\n“Hmph,” Muqwa crooned defensively. “We take the utmost care of it. For over a thousand years, our order has followed the Oracle’s instructions to the letter.”\n\n“Since when do control panels give instructions?”\n\n“I’ll admit, the Oracle’s words have been . . . sparse these last . . . well, I’ve heard it speak once. I think.”\n\n“You think?”\n\n“Yes. We’ve pieced together what we can, and we never fail in our duties. Or at least we have not failed yet. But as long as I draw breath,” he said, breathing deep and straightening his back, “I will attend the Oracle.”\n\n“Muqwa?” A thought struck Poire. Maybe it was the gas making him light-headed.\n\n“Yes?”\n\nHe hesitated. “Thank you for helping me find my way. This world is strange to me. And the people . . .”\n\n“Your presence is all the reward I could ever need.” Muqwa bowed his head.\n\nPoire swallowed. What was he supposed to say to that?\n\nWhat can you possibly see in me?\n\nBeyond the control panel, Poire saw a shape in the metal pylon: the outline of a door.\n\nJudging by all those scratches, it looked like someone had tried to open it and failed. Arcane symbols were painted over the arch in a silvery paint that glistened in the gaslights.\n\n“What do you keep in there?” Poire nodded at the door.\n\n“Ah, that is called The Door Which May Never Open. As its name would suggest, no one has ever been in there. Some say it leads to a secret gate, which leads to gods-know-where. Others say it is a symbol left here by the gods, a reminder that we are mortal and there are things we cannot do. Some paths will always be closed to us mere mortals.”\n\nMuqwa appraised the door with a wondering curiosity dancing in his eyes.\n\n“There are others still who say that a great treasure lies inside. Wonders from the gods that await their return.” Again, Muqwa gave him that expectant look, as if Poire might have all the answers.\n\nThis time, Poire thought he knew the answer. The door leads to an elevator. One of the maintenance shafts for the pylon. That’s it.\n\nBut as Poire thought about it, he realized that maybe, to these people, to Muqwa, such a simple fact could be miraculous. Even the emergency lights seemed to fascinate them.\n\nA shuffling sound stole his attention. He tuned his ears. Are those footsteps, or is the gas just getting to me?\n\n“Something wrong?” Muqwa asked.\n\n“Do you hear anything?”\n\nMuqwa tilted his head, listening to the silence.\n\n“It is quieter than normal in here. Normally, it takes three of us to tend to the lights and the offerings. Speaking of which, let me show you. This is how we speak to the Oracle. Or attempt to, at least.”\n\nCopper wires crisscrossed the floor in front of the control panel, braided over and into each other as they fanned down from the dome. They fed into crude generators, simple engines that were hooked up to open tanks.\n\nIs that gas? In open containers? No wonder it was so hard to think down here. Do they know how dangerous that is?\n\n“Awakening the Oracle,” Muqwa said, “is a very complicated, drawn-out process. It requires the utmost attention to detail. One slip, and we must begin the whole process again. First, we attune the alternators.” Muqwa gestured at the generators. “That’s the Oracle’s word for these engines. Now—”\n\nPoire put his palm on the dome. The moment his fingers grazed the chrome surface, a light sprang up. Holographic displays swirled around the dome, coming to a stop in front of him. Towering blue and black letters spelled out: “Welcome, New User.”\n\nHe heard a strangled gasp from Muqwa.\n\n“How—how did you—”\n\nAnd then the letters changed: “Tower 3 Status: Offline.”\n\nAround the corners of the display, new menu options materialized. All but one were greyed out: “Need Help?” These letters glowed faintly.\n\nMuqwa’s beak worked at empty air. A hundred emotions played across his feathery face. His eyebrows twitched and bounced, his beak opened and closed. Slowly, he sank to his knees, muttering a prayer. “By the Divine, I am not worthy. I am a worm beneath the dirt.”\n\nPoire crouched down next to him, leveling his face with the priest’s. He wanted to tell Muqwa to stop debasing himself. He wanted to show him how it worked. “It’s just a control panel. Here—”\n\n“There!” The voice echoed across the temple. It cut through the hiss of gas like a blade through flesh. “There he is! I’ve found them!”\n\nBrother Osul was crouched on the steps. And behind him, more shouts. The stomping of boots, the jingling of metal, the rushing of soldiers in uniform.\n\nPoire threw his hood back on, but it was too late. They had seen.\n\nMuqwa threw his arms out, spreading his wings to cover Poire.\n\n“You will not touch him!” he shouted, his whole body quavering with righteous fury. “You shall not lay a finger upon his Divine being!”\n\nA gunshot.\n\nMuqwa jerked and dropped to the floor like a sack of grain*.*\n\n“No!” Poire was screaming. “No!”\n\n“Don’t shoot!” another cyran shouted. “Can’t you smell the gas? Don’t shoot, damn you!”\n\nPoire fell upon the old avian, trying to press his hands against the wound as if somehow that would help. There was so much blood. How is there already so much of it? It dripped down the avian’s beak and shone on the side of his head where one of the bullets had ripped away those graying feathers and the skin underneath. It even spattered across the black chrome of the control panel.\n\n“Muqwa, can you hear me?” Poire grabbed at his robes, trying to pull him back up to a sitting position. Poire’s armor rippled and dripped through Muqwa’s feathers as if trying to help Poire hold him.\n\nA faint whisper. “Look.”\n\nOne feathered finger stretched out, pointing beyond the control panel. He was smiling. Why is the stupid fool smiling?\n\n“The door opens for you, Poire.” A gap formed in the wall of the pylons. “Just like the prophecy said . . .”\n\nThe priest’s eyes closed.\n\nUnlike the door, they would never open again.\n\nTears were streaming down Poire’s cheeks. His hands were red with blood that was not his. A dozen guards gathered around him. Those blue-and-black uniforms were just shapes, blurred by the tears in his eyes.\n\n“Grab him.”\n\nThe soldiers moved in, forming a loose circle around him, but none of them wanted to touch him. They were afraid of him.\n\n“What are you doing? Grab him, you fools! Grab him!”\n\nPoire was the first to move. He ran through a gap in their circle. Two cyrans tried to close ranks against him, but as Poire’s body collided with theirs, the liquid armor lashed out. Vicious spears of metal pierced through Poire’s clothes, ripping his cold suit and stabbing at their palms and arms, riddling them with holes.\n\nOne of the cyrans was screaming.\n\nPoire dashed up the steps and ran through the open door of the pylon.\n\nThe door slammed shut behind him. The light from the temple was gone.\n\nWrapped in the safe silence of darkness, there was nothing but the cold metal floor and Poire’s heaving gasps.\n\nA machine’s voice broke the silence. “Are you real?”\n\n***\n\nNext >\n\n***\n\nSupport from readers like you helps me keep this story going!\n\n- Buy the book here: https://pshoffman.com/books/\n\n- Become a Patron: https://www.patreon.com/pshoffman\n\n- Or just share this story with others!",
        "Agartala: The incessant rains across the Northeast are now putting pressure on states like Tripura, Mizoram and Barak valley, as the road considered the lifeline for these regions—the National Highway 6—witnessed several landslides, especially in Meghalaya.\n\nTripura, for now, seems to be the worst hit due to the disruptions across National Highway 6. Higher officials of the food and civil supplies department said if the road connectivity is restored within a few days, there would be no major effect. However, if the traffic remains disrupted for the long term, a crisis may hit the state markets. “The present stock of food grains is enough for this month. Earlier, large quantities of rice and other essential grains could be transported through railways. In absence of railway services, we are largely dependent on roadways. And definitely, the supply chain needs to remain unaffected when we have only one route operational,” Saradindu Chowdhury, secretary, Tripura Food and Civil Supplies department told EastMojo.\n\nADVERTISEMENT CONTINUE READING BELOW\n\nRailway services were snapped after heavy rains triggered landslides and inundated Haflong railway station. Expectedly, the railway operations are slated to resume on July 10. “As per the reports we have received from the National Highway Authorities, the road will be restored within the next two to three days. We are in constant touch with the highway authorities and accordingly, steps will be undertaken,” principal secretary, transport department told EastMojo.\n\nHowever, as per the sources in the food and civil supplies department, the state has a stock of essential food grains for the next ten to fifteen days while the stock of petroleum oil is enough for the next eight to ten days. “The alternative Guwahati-Agartala route through Bangladesh is still operational and IOCL is transporting gas bullets frequently. If the road connectivity also gets halted for a long time, we would be compelled to depend on the transport route through Bangladesh for transit of essentials,” said a top source in the department.\n\nMizoram, on the other hand, seems prepared for the floods better than Tripura. For now, according to people EastMojo spoke with, the state has essentials thanks to the road between Silchar and Aizawl which is still functioning. However, if the NH-6 remains out of commission, it will put much more pressure on the state, especially for fuel needs. Petrol pumps across the state witnessed more activity than usual, however, and late on Thursday evening, the state government issued a directive limiting how much petrol a vehicle can buy. According to the state directive, a two-wheeler scooter can buy 3 litres of petrol at a time, while a motorcycle can buy up to 5 litre at a time. The limit for light motor vehicles (cars) is 10 litres, and heavy motor vehicles can buy up to 40 litres at a time. The orders are effective from June 16.\n\nAlso Read: Assam battered by heavy rains for third day in row; landslide kills two kids\n\nTrending Stories\n\n\n\n\n\n\n\n\n\n",
        "At least six people were injured and three cars were trapped after a landslide dragged more than two tons of rocks onto one of the main avenues of Bolivia’s capital city, La Paz.",
        "QUIRINO — The Department of Environment and Natural Resources (DENR) Region 2 claimed that there were no traces of illegal activities in Sierra Madre.\n\nPhoto from the DENR\n\nRegional Executive Director Gwendolyn Bambalan of the DENR issued the statement after conducting an investigation to determine the cause of a landslide which covers an approximate average width of 90 meters and total height of 1,592 meters above sea level.\n\nThe landslide took place at Sitio Diariong, Barangay Villa Gracia, Maddela, Quirino.\n\n“The landslide incident was a natural occurrence due to heavy rains,” stressed director Bambalan.\n\nThe investigating team was composed of personnel from the Community Environment and Natural Resources Offices of San Isidro in Isabela and Nagtipunan in Quirino and the local government units of San Agustin and Maddela towns.\n\nBambalan added that the area is covered with mossy forest and limestone rock formation within the Quirino Protected Landscape.\n\nShe also noted that the team discovered bedrocks and boulders along the Sili Creek found at the foot of the incident area, and discolored flowing water from the top of the landslide site.\n\nBambalan also noted the absence of inhabitants or community in the area and that no traces of illegal activities were found in the site and nearby areas.\n\nThe investigators spent almost a week to reach the exact location of the landslide in the mountain.\n\nThe agency has yet to ascertain the condition of the uppermost area of the mountain and determine what possibly caused the landslide incident.\n\nThe DENR together with the Mines and Geosciences Bureau has scheduled aerial surveillance in coordination with proper authorities to identify what caused the landslide.\n\nThe DENR conducted the investigation after a netizen posted on his Facebook account photos of landslide with caption ‘Mahal naming Sierra Madre, bakit may sugat ka, sana’y huwag lumala ang sugat mo. Sanay mapansin natin ito.’\n\nThe post went viral in social media with 17,000 reactions when shared by Philippine Weather System/Pacific Storm Update.\n\nErvin Suguitan, employee of LGU San Agustin, Isabela and a freelance photographer of San Agustin Isabela took the photos on May 28 and posted them on his FB account on May 29.\n\n“Naisipan ko lang na ipost noon kasi nababahala ako kasi noong nagpakita yan, naging chocolate ang Cagayan river sa amin,” he told the Manila Bulletin.",
        "Heavy rains across the Pacific Northwest spurred concerns about minor flooding and landslides in parts of Oregon.\n\nFederal water managers are increasing water storage behind the John Day Dam in northeastern Oregon to help with water levels further down the Columbia River Basin.\n\nTHANKS TO OUR SPONSOR: Become a Sponsor\n\n“This precipitation has fallen on our mountain snowpack which has added even more water into our river systems,” said Steve Barton of the U.S. Army Corps of Engineers in a press release.\n\nBarton said several points throughout the basin will reach flood stage.\n\n“Of primary concern to us at this point is public safety,” he said. “This system of dams was built primarily to protect the public from catastrophic floods. While we cannot prevent all flooding, we can take steps to minimize the impacts of these powerful weather events.”\n\nRising water levels in the Willamette River closed a portion of Portland’s popular multi-use path, the Vera Katz Eastbank Esplanade, from the Morrison Bridge to the Steel Bridge. City officials say it could remain closed for several days. The river is expected to rise to 15.5 feet.\n\nTHANKS TO OUR SPONSOR: Become a Sponsor\n\nThe Columbia River also rose over the weekend, reaching 15.6 feet by mid-Sunday. The National Weather Service has issued a flood warning along the river from Vancouver to Longview.\n\nFlood stage in Vancouver is 16 feet (4.87 meters), and forecasters say river levels will approach that level repeatedly through late Wednesday morning. At 16 feet, minor flooding occurs, affecting islands and low-lying areas, with minor impacts on parks or trails along the river, according to the National Weather Service.\n\nThe NWS also issued a flood watch for much of northeastern Oregon through Sunday night. Officials warned people to be on the lookout for potential landslides.\n\nLandslides can occur near steep terrain, especially near burned areas from recent wildfires, and they can move faster than a person can run.\n\nMarine deputies and lifeguards are warning swimmers to stay away from local rivers for several days after immense rainfall and rapid snowmelt. The Sandy River (pictured) is especially swollen, fast-moving and numerous hazards exist. pic.twitter.com/Y3m2S4O3xx — Multnomah Co. Sheriff’s Office (@MultCoSO) June 11, 2022\n\nEmergency officials are warning swimmers to stay away from local rivers in the coming days, as the rainfall could cause river swelling and rapid currents.\n\nFirefighters in Ridgefield, Washington, rescued a man and his two daughters Saturday afternoon after their boat capsized in the Lewis River. The boat had struck a collection of trees in the river before it capsized. The man was able to hang on to his 8-year-old daughter after they were thrown into the water, but his 12-year-old daughter was sent downstream. All three were rescued from the river in stable condition. Rescue crews say the two girls were wearing flotation devices, which likely saved their lives.\n\nHeavy rain between April and June has already made it the wettest year for that period on record since 1940, according to NWS Portland. Monitors at the Portland International Airport measured 12.23 inches of rain as of June 11. The previous record was 11.87 inches in 2010.",
        '#Life in #Ankara is paralyzed! Roofs flew, #trees fell! #businessessentials flooded! While heavy #rain was #effective in Ankara, the roads turned into a lake in a #short #time; Some #homes and businesses were flooded. #Roofs were blown off and trees fell in various parts of the city. On the other hand, #Meteorology warned the region. "#Rainy #weather is expected to be effective #again after midnight tonight," the warning said. #Along with the rain, the wind was also effective in the #city. In some areas, the roofs of #buildings were blown off, trees fell. In the #Altındağ district, the windows of the building were broken in the strong wind, and the crescent on the minaret of the #mosque flew. The streets were flooded due to the problem with the manhole covers on Balgat Çetin Emeç Boulevard. While a vehicle was trapped by the rising water level, the teams mobilized to rescue it. #According to the statement made by the authorities, as of 16.00, 1378 to the fire department, #159 to Afad, more than #112 tree falling, roof flying, landslide, flooding, porthole blockage etc. notification came. The 3 trapped #children were rescued. Flood warning came, AFAD and JAK teams were directed. #flood #flooding #keralafloods #floods #flashflood #flooded #floodlights #floodlight #flooddamage #floodrelief #floodingyourtimeline #floodinsurance #floodedtimber #floodplain #floodtide #beforetheflood #dawn #weather #cloudscape #east #rainy #riverwalk #globalwarming #rain #storm #climate #disaster #disasterrelief #beautifuldisaster #naturaldisaster #disasterrecovery #disasterpreparedness #maternaldisaster #disasters #disasterresponse #disasterreliefworks #thedisasterartist #disastermanagement #naturaldisasters #masterofdisaster #nonprofit #socialgood #donate #donations #sorrynotsorry #browneyedgirl #savelives #thisisme #justme #iloveme',
        'As many as four people were killed in a landslide triggered due to incessant rain in Assam’s Guwahati on Tuesday. The heavy rains flooded the city, prompting the state government to issue an advisory asking residents not to venture outside.\n\nA huge mass of land buried a house, in which there were four people, on a hillside in Nijarapar in Boragaon area, an official of the Assam State Disaster Management Authority (ASDMA) was quoted as saying by PTI.\n\n"The landslide happened around 1 am mainly due to incessant rains. No other person is trapped as of now," ASDMA District Project Officer (Kamrup Metropolitan) Kaustav Talukdar said.\n\nRickshaw-pullers wade through a flooded street after rains, in Guwahati. (PTI Photo)\n\nThe heavy rains had led to landslides in several places in the city, including Kamakhya, Kharghuli, Hengerabari, Silpukhuri and Chandmari Colony, but there were no casualties.\n\nThe incessant rains inundated major roads across Guwahati. The approach road of the Guwahati railway station was also submerged. The flood waters have also entered hundreds of houses in the city.\n\nALSO READ | Southwest monsoon advances further to parts of Maharashtra; light to moderate rain likely in Mumbai: IMD\n\nTraffic was brought to a standstill in the city as scores of vehicles were stuck in the flood water. Massive traffic jams were reported from many areas with vehicles stranded for several hours.\n\nAssam | Waterlogging in parts of Guwahati due to heavy rainfall; hinders traffic movement, causes inconvenience to locals & commuters pic.twitter.com/PDX722PNlY — ANI (@ANI) June 14, 2022\n\nIn view of the incessant rains, the Kamrup Metropolitan district administration has issued an advisory, asking people "not to venture out unless absolutely essential".\n\n"If your residence is prone to water logging/landslides, please consider shifting to a safer location or please contact District Administration at 1077/ 86381 12297," it tweeted.\n\nRED ALERT FOR ASSAM, MEGHALAYA\n\nOn Monday, the Regional Meteorological Centre in Guwahati issued \'Red Alert\' for Assam and Meghalaya and predicted up to "isolated extremely heavy rainfall" from Tuesday to Thursday.\n\n"...widespread/widespread rainfall accompanied with thunderstorm/lightning/heavy to very heavy with extremely heavy rainfall at isolated places is very likely to continue over Northeastern states during 13-17 June 2022," the weather office said.',
        '© Provided by Sinar Daily Tenaga Nasional Berhad (TNB) - Photo: BERNAMA\n\nSEREMBAN - A landslide occurred at Jalan Tun Dr Ismail, Taman Bukit Sentosa here last night following several hours of continuous heavy rain, affecting a Tenaga Nasional Berhad substation there.\n\nSeremban Fire and Rescue Staton operations officer PGK II Jefri Uzus said a team was rushed to the scene at about 8.05 pm.\n\n"The landslide caused damage to the transformer at the sub-station. Two vehicles, a Mercedes Benz and a Volvo, were also damaged by fallen trees due to the landslide.\n\n"However, there were no casualties, but residents in the vicinity and the surrounding areas are advised to be vigilant and report any untoward incidents to the authorities immediately," he said in a statement here today.\n\nIn another incident, Seremban 2 Fire and Rescue Station operations officer PGK II Harzizi Hasim said a fire broke out at an ironing room of a boy’s hostel near here at about 2 am today.\n\nThe fire caused 40 per cent damage, but there was no casualty involved, he said. - BERNAMA',
        "The population of Belita, a village in the Kadey Division , located on the border with the Central African Republic, continued to search for the bodies of miners who disappeared on Thursday evening in an earthquake.\n\nAccording to village sources, relayed by ”Forêts et Développement Rural” (Foder), seven people were engulfed in this new gold mining site opened in Belita. Three men and two women, each carrying a baby on their back, were reportedly swallowed up. But on its Facebook page, the daily newspaper Cameroon tribune reports that the bodies of five miners have been found and eight others have been rescued.\n\nThis is not the first time that a similar tragedy has been recorded in the gold mining villages of the Eastern region. In May 2021, the collapse of two mines in Kambele, in the Batouri district, killed several dozen people, including children.\n\nBetween 2013 and April 2021, Foder recorded a total of 157 deaths at mining sites in Cameroon. Of these deaths, 25% were due to drowning and landslides caused by open pits left by miners. Of the 100 deaths whose ages are known, 38 are between 0 and 20 years old. Cameroon counts over 185 death due to gold extraction in the mines.",
    ],
)


(['mudslide',
  'landslide',
  'landslide',
  'landslide',
  'creep',
  'landslide',
  'landslide',
  'landslide',
  'landslide',
  'rock_fall',
  'rock_fall',
  'mudslide',
  'mudslide',
  'landslide',
  'debris_flow',
  'mudslide',
  'mudslide',
  'landslide',
  'landslide',
  'mudslide',
  'debris_flow',
  'debris_flow',
  'rock_fall',
  'complex',
  'landslide',
  'landslide',
  'landslide',
  'landslide',
  'landslide',
  'landslide',
  'rock_fall',
  'landslide',
  'landslide',
  'landslide',
  'landslide',
  'landslide',
  'landslide'],
 ['continuous_rain',
  'downpour',
  'downpour',
  'rain',
  '',
  'continuous_rain',
  'continuous_rain',
  'continuous_rain',
  'downpour',
  'freeze_thaw',
  '',
  'continuous_rain',
  'continuous_rain',
  'downpour',
  'continuous_rain',
  'rain',
  'continuous_rain',
  'continuous_rain',
  'continuous_rain',
  'continuous_rain',
  'downpour',
  'continuous_rain',
  'downpour',
  'rain',
  'downpour',
  'continuous_rain',
  'rain',
  'constru